In [21]:
# Generic imports
import pandas as pd
from typing import Tuple, cast
import plotly.express as px
import warnings

warnings.filterwarnings("ignore")

In [2]:
# Local imports
from thompson_sampling import PathwiseThompsonSampling
from plotting_helpers import *

In [3]:
# Ax & BoTorch imports
from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.modelbridge.generation_strategy import GenerationStep, GenerationStrategy
from ax.models.torch.botorch_modular.surrogate import Surrogate
from ax.modelbridge.registry import Models
from ax.utils.common.typeutils import not_none
from ax.core.types import TParameterization, TEvaluationOutcome, TModelPredict
from ax.core.observation import ObservationFeatures
from ax.modelbridge import TorchModelBridge
from botorch.acquisition.analytic import (
    ProbabilityOfImprovement,
    ExpectedImprovement,
    UpperConfidenceBound,
)
from botorch.models.gp_regression import SingleTaskGP

## Mining Gold!
I used [this website](https://frnsys.com/misc/polynomial_designer/) to recreate the gold content function from the OP.

In [4]:
def gold(x: float) -> float:
    return (
        2.85
        + 9.20 * np.power(x, 1)
        + -8.32e0 * np.power(x, 2)
        + 2.39 * np.power(x, 3)
        + -2.10e-1 * np.power(x, 4)
    )

In [5]:
# Gold can occur between 0 and 6.
x = np.linspace(0, 6)
# Setup our observation features to use for model predictions.
observation_features = [
    ObservationFeatures(
        parameters={"x": i},
    )
    for i in x
]
y = [gold(i) for i in x]

Plot the gold content function over our domain.

In [6]:
render_plotly_html(
    px.line(
        pd.DataFrame({"X": x, "Gold Content": y}),
        x="X",
        y="Gold Content",
        title="Ground Truth for Gold Content",
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.34.0.min.js"></script>                <div id="48b75e41-9bad-46b3-bc94-4eb4ee2a9251" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("48b75e41-9bad-46b3-bc94-4eb4ee2a9251")) {                    Plotly.newPlot(                        "48b75e41-9bad-46b3-bc94-4eb4ee2a9251",                        [{"hovertemplate":"X=%{x}\u003cbr\u003eGold Content=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"","orientation":"v","showlegend":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"xaxis":"x","y":[2.85,3.856123347536194,4.638417508253971,5.221510704359092,5.628898102467025,5.8829418136029314,6.00487089320169,6.01478134110787,5.9316361015757515,5.773265063269314,5.5563650592622364,5.296499867037907,5.008100208489416,4.7044637499195545,4.397755102040818,4.099005819975396,3.818114403255205,3.5638462958218278,3.343833886026604,3.1645765066304996,3.0314404348042556,2.948658892128277,2.919332044592691,2.9454270025973184,3.0277778209516644,3.166085498874992,3.3589179799961926,3.6037101523539157,3.896763848396514,4.233247844982003,4.607197863378097,5.011516569262348,5.4379735727217735,5.877205428253326,6.318715634763429,6.7508746355685645,7.1609198183944756,7.534955515376893,7.857953003061169,8.113750502402425,8.28505317876541,8.35343314192454,8.29932944606415,8.10204808977798,7.739762016069648,7.18951111235242,6.427202210449309,5.42760908659298,4.164372461425842,2.6100000000000705],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"X"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"Gold Content"}},"legend":{"tracegroupgap":0},"title":{"text":"Ground Truth for Gold Content"}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

## Active Learning

### Ax Helper Functions
First, define some helper functions for common Ax operations.

In [7]:
def evaluate(parameterization: TParameterization) -> TEvaluationOutcome:
    """Evaluate a parameterization on our groundtruth gold function."""
    x = cast(float, parameterization.get("x", 0.0))
    y = gold(x)
    return {"gold": (y, None)}


def create_generation_strategy(
    botorch_acqf_class, random_class=Models.SOBOL, random_trials: int = 3
) -> GenerationStrategy:
    """Strategy consisting of random search followed by surrogate/acquisition."""
    return GenerationStrategy(
        steps=[
            # Always include random steps.
            GenerationStep(
                model=random_class,
                num_trials=random_trials,
            ),
            # Combine an acquisition class with a GP surrogate function.
            GenerationStep(
                model=Models.BOTORCH_MODULAR,
                num_trials=-1,
                model_kwargs={
                    "botorch_acqf_class": botorch_acqf_class,
                    "surrogate": Surrogate(SingleTaskGP),
                },
            ),
        ]
    )


def create_client(generation_strategy: GenerationStrategy) -> AxClient:
    """Create a ax client for the gold content experiment."""
    ax_client = AxClient(
        generation_strategy=generation_strategy,
        random_seed=1,
        verbose_logging=False,
    )
    ax_client.create_experiment(
        name="gold_content",
        parameters=[
            {
                "name": "x",
                "type": "range",
                "bounds": [0.0, 6.0],
                "value_type": "float",
                "log_scale": False,
            },
        ],
        # We want to get the most gold possible.
        objectives={"gold": ObjectiveProperties(minimize=False)},
    )
    return ax_client


Cache = Tuple[Dict[int, TModelPredict], Dict[int, List[List[float]]]]


def run_ax(n: int, ax_client: AxClient, acqf_n: int = 1) -> Cache:
    """Run an ax client for a specified number of iterations."""
    predictions: Dict[int, TModelPredict] = {}
    acquisitions: Dict[int, List[List[float]]] = {}
    for i in range(n):
        # Get the next trial's parameterization.
        parameterization, trial_index = ax_client.get_next_trial()
        # Extract the model for caching this iteration.
        model = cast(
            TorchModelBridge,
            not_none(ax_client.generation_strategy.model),
        )
        # We won't always have an TorchModelBridge that supports evaluation.
        # If we do, run evaluation and cache the results for visualizing.
        try:
            predictions[i] = model.predict(
                observation_features=observation_features,
            )
            acquisitions[i] = [
                model.evaluate_acquisition_function(
                    observation_features=observation_features
                )
                for _ in range(acqf_n)
            ]
        except Exception:
            pass
        # Complete the trial by evaluating the true gold content.
        ax_client.complete_trial(
            trial_index=trial_index, raw_data=evaluate(parameterization)
        )
    return predictions, acquisitions

Store results of each optimization so we can compare the different methods at the end of this section.

In [8]:
results: Dict[str, pd.DataFrame] = {}

Sample uniformly from the domain to show how we learn the gold content with our surrogate function. Our surrogate function will be a [Gaussian Process](https://cs229.stanford.edu/section/cs229-gaussian_processes.pdf):

$$
\begin{align*}
  \begin{bmatrix}
    f(x_1) \\
    \vdots \\
    f(x_m)
  \end{bmatrix}
  &\sim \mathcal N \big(\begin{bmatrix}
    m(x_1) \\
    \vdots \\
    m(x_m)
  \end{bmatrix}, 
  \begin{bmatrix}
    k(x_1, x_1) & \dots & k(x_1, x_m) \\
    \vdots & \ddots & \vdots \\
    k(x_m, x_1) & \dots & k(x_m, x_m)
  \end{bmatrix} \big) \\
m(x) &= \mathbb E[x] \\
k(x, x') &= \mathbb E[(x - m(x))(x' - m(x'))] \\
y^{(i)} &= f(x^{(i)}) + \epsilon^{(i)},\quad i=1,\dots,m, \quad \epsilon^{(i)}\sim N(0, \sigma^2)
\end{align*}
$$

In [9]:
generation_strategy = create_generation_strategy(
    botorch_acqf_class=ProbabilityOfImprovement,
    random_class=Models.UNIFORM,
    random_trials=5,
)
ax_client = create_client(generation_strategy=generation_strategy)
predictions, acquisitions = run_ax(n=10, ax_client=ax_client)

[WARNING 08-10 23:19:20] ax.service.ax_client: Random seed set to 1. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 08-10 23:19:20] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x', parameter_type=FLOAT, range=[0.0, 6.0])], parameter_constraints=[]).


Plotly can be animated by adding a [slider](https://plotly.com/python/sliders/) to the figure layout. Note, the first couple points are the uniformly random samples. These are followed by alternating the surrogate function and a query point.

In [10]:
fig = go.Figure()
# Plot our groundtruth function for comparison.
fig.add_trace(plot_groundtruth(x=x, y=y, kwargs=get_line_kwargs()))
# Iterate through the trials.
n = len(ax_client.generation_strategy._generator_runs)
j = len(fig.data)
for i in range(n):
    # Not every iteration has predictions.
    if i in predictions:
        fig.add_trace(
            plot_predictions(x=x, predictions=predictions, i=i),
        )
    # Finally, add where we observed this iteration.
    fig.add_trace(
        plot_observed(
            ax_client=ax_client,
            gold_fn=gold,
            i=i,
            kwargs=get_scatter_kwargs(),
        )
    )
# Create the animation steps.
steps = [
    dict(
        method="update",
        label=fig.data[i].name,
        args=[
            {
                "visible": [True] * (j + i) + [False] * (len(fig.data)),
                "title": str(i),
            }
        ],
    )
    for i in range(len(fig.data))
]
# Format the layout of the plot
fig.update_yaxes(range=[1, 9])
fig.update_xaxes(range=[-1, 7])
fig.update_layout(
    title="Gold Search",
    xaxis=dict(title="X"),
    yaxis=dict(title="Gold Content"),
    # Add the animation steps onto the layout.
    sliders=[dict(active=0, steps=steps)],
)
render_plotly_html(fig)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.34.0.min.js"></script>                <div id="a66f5376-e01a-4215-8e10-fa36ac3dcd4d" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("a66f5376-e01a-4215-8e10-fa36ac3dcd4d")) {                    Plotly.newPlot(                        "a66f5376-e01a-4215-8e10-fa36ac3dcd4d",                        [{"hoverinfo":"skip","mode":"lines","name":"Ground Truth (f)","showlegend":true,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[2.85,3.856123347536194,4.638417508253971,5.221510704359092,5.628898102467025,5.8829418136029314,6.00487089320169,6.01478134110787,5.9316361015757515,5.773265063269314,5.5563650592622364,5.296499867037907,5.008100208489416,4.7044637499195545,4.397755102040818,4.099005819975396,3.818114403255205,3.5638462958218278,3.343833886026604,3.1645765066304996,3.0314404348042556,2.948658892128277,2.919332044592691,2.9454270025973184,3.0277778209516644,3.166085498874992,3.3589179799961926,3.6037101523539157,3.896763848396514,4.233247844982003,4.607197863378097,5.011516569262348,5.4379735727217735,5.877205428253326,6.318715634763429,6.7508746355685645,7.1609198183944756,7.534955515376893,7.857953003061169,8.113750502402425,8.28505317876541,8.35343314192454,8.29932944606415,8.10204808977798,7.739762016069648,7.18951111235242,6.427202210449309,5.42760908659298,4.164372461425842,2.6100000000000705],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=1","opacity":1.0,"showlegend":true,"visible":false,"x":[0.32872711926390363],"y":[5.057664805217066],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=2","opacity":1.0,"showlegend":true,"visible":false,"x":[1.4348541948065263],"y":[5.091532753266253],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=3","opacity":1.0,"showlegend":true,"visible":false,"x":[1.1448646473418669],"y":[5.703244208949001],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=4","opacity":1.0,"showlegend":true,"visible":false,"x":[4.012995102221842],"y":[5.776814007863983],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=5","opacity":1.0,"showlegend":true,"visible":false,"x":[2.451072738768894],"y":[3.0295919107202014],"type":"scatter"},{"fill":"toself","hoverinfo":"skip","name":"p=6","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[5.07316634384684,4.999504522910488,5.032634654794059,5.164273638531742,5.3575527850944376,5.5568489891338,5.711965298159969,5.793407593601372,5.793046710584509,5.714025272511392,5.5600243071368824,5.337923204662083,5.070524245665279,4.788961815910491,4.509782877049986,4.231677995788834,3.9480319023640744,3.6620736253279857,3.39643006646367,3.192706536745609,3.099594778508046,3.1508663930297627,3.3459010613785805,3.6534271369530398,4.026144048242655,4.413922289654745,4.773380320739256,5.073826644284898,5.300281527146908,5.454264163752668,5.552622538439179,5.624192584556933,5.703762590807438,5.822971404330607,5.99911603888284,6.229694457181241,6.498491485685141,6.783974911678316,7.065474533162904,7.326647652755239,7.55671286607771,7.750218586592451,7.906049409724522,8.02618143070912,8.114497290043163,8.175813139212142,8.215163557391369,8.23732952209914,8.246566238545071,8.246479913608551,2.1038117576686997,2.199054270314816,2.3146515175484197,2.454304178794718,2.6219121269328403,2.821286080558268,3.0556775521343984,3.3270771103308348,3.6352434209155113,3.9764615523321556,4.342102095479228,4.717177358659123,5.079277254857443,5.398508766827107,5.639314402783944,5.765194097747739,5.747192701884677,5.576172607972804,5.273003013151644,4.881693970881187,4.454369317829374,4.039878164752305,3.6767265690557367,3.3894657457024477,3.1876287569198007,3.0666662403321716,3.010814888092022,2.998176501454861,3.0081848661092496,3.030702344003669,3.073740034884545,3.162093953009095,3.322827933956939,3.569615051916137,3.894282847692029,4.267682326786479,4.649030655275435,4.999768614540697,5.294025964716555,5.515119030153113,5.643537656434499,5.6682908988889835,5.606217448075431,5.493531441859911,5.36459781712188,5.234645610425719,5.094845010060559,4.922132176436568,4.698442022644098,4.421796493248411],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=6","opacity":1.0,"showlegend":true,"visible":false,"x":[4.194693878482701],"y":[6.431018758151026],"type":"scatter"},{"fill":"toself","hoverinfo":"skip","name":"p=7","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[5.011682324084294,4.967292420626053,5.025202045358007,5.167691506803046,5.35759903216066,5.547168449602896,5.694598673597294,5.773959136627116,5.775509317842225,5.699511018224228,5.550181987100491,5.3373942847125,5.082027050882892,4.808793912710115,4.531893404446643,4.253455542677204,3.971746765137313,3.6912837140884633,3.4293691920870373,3.2157764648311256,3.0846628857866976,3.060292735828231,3.1446283048135526,3.3187977990490745,3.5524082396772885,3.8134444764344906,4.076296630963971,4.326743674647874,4.563468597947743,4.796146785075203,5.0405339217118525,5.311499066013494,5.615756306118919,5.947097660441558,6.2881466518593205,6.624132963507047,6.954400881726091,7.285305283663272,7.6214249824415825,7.962303750557356,8.302792271205293,8.634917506364527,8.949959232100325,9.24008618238094,9.499354068036084,9.724112888748968,9.91297272384869,10.066494452268351,10.186747760928867,10.276840394120928,2.4965911970642187,2.813154012031427,3.16428410971422,3.5483202271228196,3.961176468593514,4.395693472714459,4.841050846766807,5.282377259414732,5.700747757610311,6.073802111417138,6.377227999976317,6.587287765666102,6.6843713658020985,6.657173596066782,6.506507431817513,6.2471257817942165,5.905170436837344,5.506829751551806,5.071205977287646,4.617833080232228,4.171542923542104,3.7606068620992783,3.4116167660073353,3.1440573572851385,2.966415397344681,2.874810571196833,2.8545611925303187,2.8846572335391953,2.944583349133419,3.022110487173903,3.119394142106782,3.2520810268354223,3.438733673862437,3.6890795552786795,3.9973051345597783,4.342174713201126,4.6932464960203255,5.020030222256385,5.29868887712577,5.510632776542448,5.63786040625332,5.671791673736523,5.624734746493644,5.523755297786458,5.396327195188752,5.257711648848211,5.105803731317368,4.925695757685748,4.701521788073669,4.4257313336465565],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=7","opacity":1.0,"showlegend":true,"visible":false,"x":[4.491198145517794],"y":[7.41938481136026],"type":"scatter"},{"fill":"toself","hoverinfo":"skip","name":"p=8","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[4.688935158788733,4.757263160615211,4.9151940262942935,5.134072034212077,5.374241300214174,5.593106914798788,5.756281377409039,5.842577941386738,5.842147001903871,5.751541476554139,5.5718213954975475,5.314564636658083,5.00737239981946,4.684544268081935,4.370780256336546,4.076644725009364,3.8035766609236195,3.5522523872648795,3.3294254705432156,3.149906502890123,3.032564015291765,2.9916573240373903,3.028825376678319,3.1330456504453497,3.2862045300608713,3.469976102799417,3.6716546641515455,3.887435044406619,4.122454519809857,4.3877050809853815,4.694703266390146,5.049558869425854,5.448686085515852,5.878452710084925,6.319765515819786,6.754196286926548,7.163335220172872,7.532886751931623,7.862797880872357,8.1661554444958,8.461075522557168,8.764060951480399,9.086359390352856,9.432854925393542,9.802622312785426,10.190333919155057,10.587923474378039,10.986136097637678,11.375776231328494,11.748591955594595,4.59768284060668,5.209894091630055,5.810774805544268,6.382891156057303,6.906816783258837,7.3621427802900765,7.728961711771088,7.9898044823152725,8.13189367060431,8.149409387174314,8.045256675324776,7.831632288981851,7.528656221367233,7.160756628149617,6.751172605735728,6.317848788216408,5.876645512312088,5.439517660355619,5.010475483930592,4.591987112990491,4.1915487085287735,3.8229928033293814,3.5034571857165453,3.2483785741252547,3.0668035070187454,2.9586888161259077,2.9150933476311685,2.9213679765697833,2.9626394166254744,3.030046001970331,3.125336023268469,3.259945114954613,3.4468845744693204,3.691967901689031,3.9892409555014274,4.321889365645164,4.667455647787947,5.003948810056101,5.311901565631748,5.569224984381501,5.747870468067624,5.825894449897707,5.802356667900556,5.695999817585351,5.534406224061591,5.341540169415405,5.129189423455499,4.895517371157193,4.630674455417304,4.324414590323383],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=8","opacity":1.0,"showlegend":true,"visible":false,"x":[4.608163665936905],"y":[7.746444938096204],"type":"scatter"},{"fill":"toself","hoverinfo":"skip","name":"p=9","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[4.575519201693577,4.699806649386003,4.896328546588739,5.136835688306437,5.385773660386025,5.606425046981714,5.7689602366772075,5.853946860038738,5.851156268240532,5.756645032505787,5.572493884077128,5.311997802970214,5.002473230806389,4.67617354546572,4.357154443142613,4.057822317331819,3.782586648183381,3.534040272174998,3.3181571808604344,3.1460216601573534,3.0312398995763146,2.9840215257076594,3.0056842698339796,3.0885854992390094,3.220056788124329,3.387377482180945,3.5819741546033828,3.8016384897876683,4.050228121852698,4.335009305574577,4.662472288201293,5.034037006739993,5.443415333198203,5.8771659540134555,6.318600859707222,6.75196570878708,7.161756942537717,7.534305503211123,7.860976562683477,8.141838219282375,8.387129935215114,8.613459387302207,8.838916274194304,9.0792567892431,9.345706313713913,9.644210060570536,9.975718374923915,10.337079607800096,10.722193280060106,11.12317993510144,5.935293020410067,6.520710120715554,7.0549996632607055,7.522193708343121,7.907475932189097,8.198497713938556,8.386758622939968,8.468832242469528,8.447120651453567,8.329748501336947,8.129230459099285,7.859799820448051,7.53401952143703,7.161142056785434,6.751374173881461,6.318215875875912,5.876822186256043,5.4405752735292,5.016216468019225,4.6074962895502765,4.220576348127558,3.865906800226299,3.5566186817383874,3.3050120575533417,3.1189398254305107,2.9995282631109097,2.9410792640093435,2.933318250987459,2.965448535463811,3.0307917913356035,3.130186480406596,3.2713252402945034,3.462844067733325,3.7078649057968507,4.000554035137165,4.3266393852185585,4.666983811522136,5.001656137906867,5.31093830175758,5.571525297354402,5.75568147790567,5.842191433453518,5.828216916822813,5.728212769978933,5.565623985989545,5.363000540930447,5.134494173767781,4.883303413258469,4.604525192950797,4.290545060042975],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=9","opacity":1.0,"showlegend":true,"visible":false,"x":[4.699603014745402],"y":[7.964008751168905],"type":"scatter"},{"fill":"toself","hoverinfo":"skip","name":"p=10","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[4.554023286522054,4.6894248140034165,4.893371282488958,5.137844228155514,5.3881694273653435,5.608794782911614,5.770798868873624,5.85516432778863,5.851724952661134,5.756591964706958,5.572116195742437,5.311838811326354,5.002819296471501,4.676700681543766,4.357240361677949,4.057089882233097,3.7812081005024885,3.5326311538921042,3.317345668051653,3.145945154634501,3.0312714021522855,2.982958143090487,3.0023474672819073,3.0823968463139284,3.211306960327825,3.377147114821844,3.5717817493399875,3.792956274087117,4.0440306071095655,4.331506000414363,4.661125523639355,5.033875420737961,5.443528141051006,5.877137796448127,6.318610591196382,6.752408445607565,7.162662276861828,7.534546574258889,7.856879717904279,8.125702356901952,8.347877643233026,8.538428281527253,8.716348693460956,8.90043264998519,9.106212641065138,9.344322948960112,9.620083328812274,9.933935446416715,10.2823681088751,10.659039320328425,6.27844642083621,6.838197908976841,7.336492952235262,7.759293069005221,8.094721105632107,8.334279974014873,8.473935933703496,8.51479308851488,8.463018314518528,8.328677483379202,8.12331532253288,7.85663581216383,7.534243818020564,7.162087169947465,6.751797281449238,6.31819317767644,5.876761999238586,5.440646239867888,5.016658710756508,4.608847577000458,4.223462513482099,3.8706690243188526,3.563045284099769,3.312311428522833,3.125961577281321,3.005146496582162,2.9445998096114625,2.9347580880257307,2.965560233125756,3.0307735266085873,3.1311365710383186,3.273708665000218,3.4663398951172533,3.7116355303301907,4.003727652102242,4.328728559845956,4.668015928850581,5.0019620438802255,5.310807871940873,5.571166350775728,5.755595712473674,5.843236506809652,5.831022433483485,5.732600662125818,5.570491618065869,5.366717266061792,5.135570497466749,4.880951181295884,4.598784924678657,4.281999099036873],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=10","opacity":1.0,"showlegend":true,"visible":false,"x":[4.790397405221612],"y":[8.139446456560734],"type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"yaxis":{"range":[1,9],"title":{"text":"Gold Content"}},"xaxis":{"range":[-1,7],"title":{"text":"X"}},"title":{"text":"Gold Search"},"sliders":[{"active":0,"steps":[{"args":[{"visible":[true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"0"}],"label":"Ground Truth (f)","method":"update"},{"args":[{"visible":[true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"1"}],"label":"i=1","method":"update"},{"args":[{"visible":[true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"2"}],"label":"i=2","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"3"}],"label":"i=3","method":"update"},{"args":[{"visible":[true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"4"}],"label":"i=4","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"5"}],"label":"i=5","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"6"}],"label":"p=6","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"7"}],"label":"i=6","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"8"}],"label":"p=7","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"9"}],"label":"i=7","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"10"}],"label":"p=8","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"11"}],"label":"i=8","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"12"}],"label":"p=9","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,true,true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"13"}],"label":"i=9","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"14"}],"label":"p=10","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"15"}],"label":"i=10","method":"update"}]}]},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

## Bayesian Optimization

### Probability of Improvement (PI)
Choose next point based off the highest probability over the current max.


$$
\begin{align*}
x_{t+1} &= \arg\max_x P(f(x) \geq (f(x^+))) \\
& = \arg\max_x \Phi\big ( \frac{\mu_t(x) - f(x^+)}{\sigma_t(x)} \big )
\end{align*}
$$

In [11]:
generation_strategy = create_generation_strategy(
    botorch_acqf_class=ProbabilityOfImprovement
)
ax_client = create_client(generation_strategy=generation_strategy)
predictions, acquisitions = run_ax(n=10, ax_client=ax_client)
results["PI"] = ax_client.get_trials_data_frame()

[WARNING 08-10 23:19:21] ax.service.ax_client: Random seed set to 1. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 08-10 23:19:21] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x', parameter_type=FLOAT, range=[0.0, 6.0])], parameter_constraints=[]).
[WARNING 08-10 23:19:22] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


In [12]:
render_plotly_html(
    plot_surrogate_and_acquisition(
        x=x,
        y=y,
        ax_client=ax_client,
        gold_fn=gold,
        predictions=predictions,
        acquisitions=acquisitions,
        line_kwargs=get_line_kwargs(),
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.34.0.min.js"></script>                <div id="9ea7e7ce-d63b-4207-8fb0-b902b770a79d" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("9ea7e7ce-d63b-4207-8fb0-b902b770a79d")) {                    Plotly.newPlot(                        "9ea7e7ce-d63b-4207-8fb0-b902b770a79d",                        [{"hoverinfo":"skip","mode":"lines","name":"Ground Truth (f)","showlegend":true,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[2.85,3.856123347536194,4.638417508253971,5.221510704359092,5.628898102467025,5.8829418136029314,6.00487089320169,6.01478134110787,5.9316361015757515,5.773265063269314,5.5563650592622364,5.296499867037907,5.008100208489416,4.7044637499195545,4.397755102040818,4.099005819975396,3.818114403255205,3.5638462958218278,3.343833886026604,3.1645765066304996,3.0314404348042556,2.948658892128277,2.919332044592691,2.9454270025973184,3.0277778209516644,3.166085498874992,3.3589179799961926,3.6037101523539157,3.896763848396514,4.233247844982003,4.607197863378097,5.011516569262348,5.4379735727217735,5.877205428253326,6.318715634763429,6.7508746355685645,7.1609198183944756,7.534955515376893,7.857953003061169,8.113750502402425,8.28505317876541,8.35343314192454,8.29932944606415,8.10204808977798,7.739762016069648,7.18951111235242,6.427202210449309,5.42760908659298,4.164372461425842,2.6100000000000705],"type":"scatter","xaxis":"x","yaxis":"y"},{"marker":{"size":10},"mode":"markers","name":"i=1","opacity":1.0,"showlegend":true,"visible":true,"x":[1.8140525221824646],"y":[4.153277647970732],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=2","opacity":1.0,"showlegend":true,"visible":true,"x":[5.636016067117453],"y":[6.4030358079544385],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=3","opacity":1.0,"showlegend":true,"visible":true,"x":[4.097510201856494],"y":[6.082020053347733],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=4","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.04806715789974354,0.038950223365649135,0.030354336396250188,0.022509385582449165,0.01565484471444161,0.010008710974654726,0.005719485323507537,0.0028074414473676745,0.0011187110567908807,0.00033445474097389756,6.763593052259173e-05,8.266832959809783e-06,5.831695418172122e-07,2.985012540800117e-08,2.491202025783452e-09,1.1707468232903512e-09,5.124717396995852e-09,8.585568213060402e-08,1.659421144300826e-06,2.0170013641916788e-05,0.0001388493685416449,0.0005912252445761788,0.0017531414190953476,0.0039971044449773774,0.007537595566893882,0.012382573513050206,0.018389243070984225,0.02537329167205872,0.033241721103364125,0.04215696674967249,0.05276737552157317,0.06653361719863721,0.08605825133743371,0.11490525041123252,0.1558616533909716,0.20776285815694495,0.26457331412072066,0.3189376814982125,0.366019099282582,0.40418801632713064,0.43375533297023466,0.4556737048276599,0.4707950554849367,0.479576388530276,0.4821189283174623,0.4785898253655618,0.46994527950710946,0.4582178737478434,0.44568195167095287,0.4338066248492031],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=4","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[5.879828741772096,5.723684944424338,5.564919034717484,5.405287615415519,5.246875209974377,5.092092146564724,4.943655173439945,4.804546034740408,4.677943045572257,4.567120898175266,4.4753146866562545,4.405545664614189,4.360408768005911,4.341825638785639,4.350771871227595,4.386993354512391,4.448748389221656,4.532810844813912,4.634893013277516,4.750143492373074,4.873588620047045,5.000504620600418,5.12671443494951,5.248806696645397,5.364276715061822,5.471590800773641,5.570176158374724,5.660339316717575,5.74311715203231,5.820066490320642,5.893001544577461,5.96369348171465,6.03355350613943,6.103329968324751,6.172860705100469,6.24095696127736,6.305593589211606,6.364345953453986,6.4148589511706895,6.45527159344706,6.484555617589533,6.502739129116825,6.51099724395559,6.511601248774515,6.50772643883592,6.503127171113509,6.501696264117567,6.50693847950454,6.521501446900843,6.547022552377477,6.101743942952618,6.170066285289676,6.219591383202151,6.250977680328067,6.2659653602492105,6.267250979972175,6.257994311081833,6.241256977277889,6.219539143177324,6.194444308974291,6.166489252021625,6.135068025432309,6.098571020386865,6.054651956833938,6.000626656609064,5.933977068904017,5.852922089199586,5.75696588950724,5.647101205867781,5.525598774378168,5.395669412032708,5.2610911299200485,5.125838350762745,4.993740002771914,4.868184535225848,4.751883097110591,4.646697269002982,4.553534562066633,4.472312966795951,4.401994655091324,4.340688005214141,4.285815939106026,4.234346690290113,4.183080205450835,4.128979247612083,4.069527932593617,4.003040362354105,3.928733988138434,3.8466290758021366,3.757397508226621,3.662195051125944,3.5624967627493516,3.4599483111916736,3.356240325414143,3.2530086498943227,3.1517603858862326,3.053823678413531,2.960318116876881,2.872142141022751,2.789973795802151],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=4","opacity":1.0,"showlegend":true,"visible":false,"x":[5.376579210943656],"y":[7.78026814236398],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=5","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.03270227567121766,0.02699984846732462,0.021605335096855312,0.016645775335546786,0.01224713736324175,0.008519159826573702,0.005535693919053022,0.003314228817432878,0.0018021351608542668,0.0008794277365751368,0.00038397629851540864,0.00015265943848391807,5.8378307113452185e-05,2.388250209303332e-05,1.2153796514674943e-05,8.90432277240389e-06,1.0011264030674819e-05,1.629988932331542e-05,3.346074355703932e-05,7.492737417722573e-05,0.000164820796343754,0.0003363574213714318,0.0006229168460518576,0.0010450875170578175,0.0016011853600457527,0.002266640928120571,0.0030023685229162646,0.0037686228199229313,0.004540322979693665,0.005321129056209206,0.006155246125739249,0.00713721089258916,0.008419872293406717,0.010217038120916306,0.012786108561788233,0.016363692147031886,0.021057945463689984,0.02674066846041739,0.03299832108694814,0.039201945586391396,0.044701388848710284,0.049093623527079105,0.05248475809260955,0.05563412203312857,0.05983911035537887,0.06648225453615765,0.07638454912165019,0.08933423717327743,0.10416035012235633,0.11935167318622429],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=5","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[7.495651257760675,7.284834873266803,7.066648672973573,6.843106579207252,6.616719721149,6.390513858849745,6.168023792794874,5.953256833214429,5.750616697632555,5.564779038074719,5.400510484652241,5.262425011981605,5.154675016004324,5.080580144516059,5.0422049366001085,5.039906630651957,5.071909162410016,5.134264322955848,5.221450663302278,5.327092357566368,5.444597336589617,5.567694335321278,5.69085997443865,5.80963297282226,5.920816446145916,6.022571306179688,6.114404639850731,6.197057358718479,6.272296172046842,6.342616873323181,6.410869842905653,6.479825268191519,6.551705345010836,6.627723635688087,6.707687178174618,6.789787685359177,6.870937771331512,6.947534691565976,7.0162211970593304,7.074519720757224,7.121287600760339,7.156963394743796,7.183594938519506,7.2046588613056555,7.224699072361968,7.248825258084782,7.282077289406204,7.328642314700801,7.391199689415242,7.4708353553172735,6.244371433637564,6.380361619600279,6.4857553812090085,6.558925988108057,6.599590092991258,6.608925920272547,6.5895898562198,6.545575313341296,6.481545809613815,6.402025221171347,6.310774782550063,6.210403359288035,6.1022279125734675,5.986384732490723,5.8621741298915735,5.728601597092776,5.5850568085970735,5.43200750549391,5.271317815118637,5.106074215983993,4.940227463057346,4.778168188253633,4.624291483562842,4.482590301250648,4.356304463717433,4.247642149406956,4.157583920461708,4.085775199382301,4.030510869208282,3.9888145543540365,3.9566143039828385,3.9290150310244236,3.9006654014728843,3.8662122993906496,3.8208291753302377,3.7607934114317016,3.68396730695931,3.589827927557511,3.4791971757993863,3.353918972676915,3.216537498561446,3.070003181522298,2.917421925272162,2.7618543050479074,2.60616517081994,2.452919983933062,2.304321847045188,2.1621821280418287,2.027917433249126,1.902566130487723],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=5","opacity":1.0,"showlegend":true,"visible":false,"x":[6.0],"y":[2.6100000000000705],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=6","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.08653692470322279,0.08156056891101435,0.07625600182025259,0.07065389706779239,0.06480464613287185,0.0587824508630282,0.05268861264445879,0.04665279448677954,0.04083065378889843,0.03539627440546545,0.03052878913022684,0.026394888010511496,0.023132125566322995,0.020840002602230068,0.019583400019066766,0.019404299698161744,0.020327254734751176,0.02235412696439887,0.025455591003541878,0.029553476554349194,0.034507519618515815,0.040118901403697116,0.04614988225923302,0.05235081942124289,0.05848558461157401,0.06435048346958634,0.06978598681967643,0.07468283604617043,0.07898427535806345,0.08268491609948829,0.08582466436187419,0.08847384204779118,0.09070411766668868,0.09254093313601472,0.09389891475116421,0.09451989938665824,0.09397788859387556,0.09176573020313501,0.08741953273970508,0.08066118295517602,0.07154360270953186,0.06056901643391124,0.048715657369874474,0.03728086441426747,0.02750731428417421,0.02015074518290084,0.015299190652774032,0.012574766288612716,0.011489487218403602,0.011674051234425921],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=6","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[10.101835960966621,9.888198543890951,9.658238360386132,9.413302160952702,9.155530794462443,8.887974501659706,8.614685118088907,8.340767220703402,8.072365901422021,7.816564956560724,7.581166724830069,7.374324846263104,7.204005499591133,7.077263151984627,6.999335443308955,6.972589683391676,6.995440643965557,7.062060768490481,7.1634897961014286,7.289016474941914,7.427384067775535,7.567778983479824,7.700595295745157,7.817984728305926,7.914209922374786,7.985820110038006,8.031664922398372,8.052756291079934,8.051982554324107,8.033675252211943,8.003030056982594,7.965391271459723,7.925426649291417,7.886247531698084,7.848568726133738,7.810169967018549,7.766448613141656,7.711849341365584,7.641270062929096,7.551206099982584,7.440549475902452,7.311004421531266,7.167118419863434,7.015963409686333,6.866537131647981,6.728990922048703,6.613818246371373,6.531031681013244,6.489307994101274,6.495224227837332,3.815503019839656,4.0809396964228135,4.304558188312066,4.482725761700065,4.6138023051215145,4.69871999165364,4.741075711789359,4.7468742272226665,4.723370563258119,4.677625501050517,4.615369116335052,4.5402741587391775,4.453684452510247,4.354813161767793,4.2413946899107575,4.110739339958814,3.961098652757516,3.7930898458419375,3.6102244965341526,3.4183176537896034,3.2246115332378413,3.036878561704627,2.8625930748560178,2.708222418217258,2.5786606598239565,2.476811301509039,2.403317709886214,2.356439183734948,2.3320742021732315,2.32393809210235,2.3239078118235215,2.322549462152106,2.3098421621876515,2.276102815258609,2.213098356695345,2.115298577007507,1.980875611665316,1.8114876852392126,1.6113535627061921,1.3863243960311729,1.1430722616705222,0.8884345636341697,0.628924339739577,0.3703968690397721,0.11785164114969238,-0.12465624018448196,-0.3540217596326851,-0.5680372487365588,-0.76530213301512,-0.9451120541456515],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=6","opacity":1.0,"showlegend":true,"visible":false,"x":[4.298252484395651],"y":[6.794103154945049],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=7","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.08966711190983648,0.0842240898112912,0.07825853154207962,0.07177966454016985,0.06482502050886371,0.057471192200251174,0.04984554335982533,0.04213629767798683,0.03459592091769693,0.027529941445981625,0.021262858109987817,0.01607959916941694,0.012159259025743625,0.00953995465955167,0.008152871830216972,0.007918771523477896,0.008839496903553465,0.011015960964133287,0.014578439364465956,0.019561140321556557,0.025811943807240504,0.032994106460401525,0.040659041214327014,0.04833394718545018,0.055585606056830994,0.06205265534159303,0.06745679517776089,0.07160790895689552,0.07441660938663533,0.07592502427606128,0.07636223734110324,0.07621890092762741,0.07630586653781592,0.07770768284392211,0.08148684044486078,0.08805973531424963,0.09656662879266104,0.10478354100566839,0.10982047848773038,0.10913104532755497,0.10126638168116592,0.08634064086687995,0.06635092503035518,0.045083942060355674,0.0269096841154058,0.014535677176743566,0.007746027514372361,0.004623898782483069,0.0034354351972235566,0.0032671894432677135],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=7","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[10.359563100177485,10.127498033751957,9.869344942840137,9.585324437239965,9.276787700662512,8.946513041659006,8.599000469567262,8.240730341349234,7.8803377118391955,7.528638211620608,7.198426139270717,6.903954599409793,6.660006520389638,6.480481930894404,6.376470129328743,6.353853621898401,6.410725134348763,6.536579382788979,6.714734303173941,6.925328685900739,7.147836799271177,7.363026640280124,7.554386998026512,7.7090932524243625,7.818595449166712,7.878902850195951,7.890615194159141,7.858719575410986,7.792139495798914,7.702995435984315,7.605521195333904,7.514585763715643,7.443806724663824,7.403318389337388,7.397383122698253,7.422426568058149,7.467116795663586,7.51458311380571,7.54606007108933,7.544655547871142,7.4982472083196905,7.401257324841062,7.255257660480019,7.068510485358868,6.854702789775962,6.631279530868436,6.417798762565178,6.2343236439524095,6.099718106292093,6.029899921350274,3.84371956224354,4.220445870713744,4.552034818879386,4.832622361275633,5.058917118936161,5.231691829150014,5.356270455073464,5.442191355741292,5.500592805710253,5.5408318477811465,5.567916775337429,5.581126012907654,5.573993431210562,5.535708846895811,5.453844626170821,5.318117421568003,5.123909996012092,4.873960345330927,4.57809652206335,4.251272874138155,3.9112510570126013,3.576450281151235,3.2641486951690046,2.989094362389832,2.762508161283289,2.5914253600409713,2.4783173897096704,2.4209503871583404,2.4124639050219296,2.44168481732228,2.493721681855289,2.550907271036703,2.594164317608307,2.604853306096338,2.567113089318842,2.4706051044974533,2.3127094092004232,2.0978559072765672,1.8352404096721253,1.5366565191912565,1.214727978718305,0.8816049333962352,0.5481056658823338,0.22323921050011553,-0.08597458769778488,-0.3744718152207849,-0.638972847083898,-0.8777331072926522,-1.0902620803742984,-1.2770426796645715],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=7","opacity":1.0,"showlegend":true,"visible":false,"x":[4.702762526338227],"y":[7.97083381305886],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=8","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.15521959126548893,0.15300862259666823,0.1498826857563066,0.1455675425534418,0.13971873073080796,0.13190784693637497,0.1216136605503268,0.10823421265700107,0.09116477917877698,0.07006016762213087,0.04556540246919751,0.020980694301082417,0.004047759051219702,3.605164637933026e-05,4.1567441753608643e-17,1.0593636629076215e-98,6.660105530183806e-10,0.00023876872362523017,0.0055125785760729825,0.01914927513793499,0.03524630973460835,0.048846504410500015,0.05798188133899445,0.06222820190091103,0.061730239720895516,0.056823128632072076,0.047987480091922316,0.036028498241639076,0.022468365445896678,0.010020481326018146,0.002211174097759848,8.154814061963776e-05,8.094382155404229e-09,1.0184514870575887e-23,2.0735509286007857e-43,8.201398951485256e-20,8.181237148992037e-07,0.006622890189304626,0.22018749303211074,0.7766985776605184,0.8719450775495452,0.8828395506071974,0.8747505088602466,0.7675836036468769,0.07706053288242017,8.632509787514815e-10,5.5063984338751604e-31,2.3751138495285948e-58,1.0399303993859618e-129,1.2320065118431594e-270],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=8","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[21.676767258261275,21.13005010660933,20.454106138339977,19.630228382844418,18.64188555062205,17.477365970523742,16.13325566358391,14.618691472232442,12.960115697139818,11.205929377755103,9.429988090309912,7.732334911334418,6.235019858489645,5.070530931631973,4.360634453255586,4.1848129200711845,4.542246634697611,5.338225903338655,6.417702143666762,7.605027521121874,8.733373261191389,9.663690634564004,10.295251260791769,10.570328353161335,10.475208035048325,10.038880107409607,9.32975099367756,8.449759413513647,7.5245633818101565,6.688294829562247,6.0621854960281745,5.728730527976311,5.707443981439658,5.944233517399304,6.330578464923357,6.761760686418145,7.185747196804042,7.5652248362256325,7.878459749674508,8.141286956072545,8.34975529575322,8.449524112601505,8.391164171796682,8.16947882569555,7.79040776259195,7.212189738273538,6.372883189533406,5.2755655560944,3.994941501785359,2.6816171467164875,2.635883356614943,3.9402241307523242,5.218037056351114,6.333415035215678,7.179042556707615,7.750958576386673,8.080508893850348,8.220960904628257,8.24771616703707,8.204341616494432,8.075807039953354,7.8345431375730294,7.5030683035955485,7.128094420820124,6.724842433250822,6.301676422582003,5.8580666113226805,5.330618521492264,4.633876754104501,3.723138711398944,2.6185036646415423,1.3976413945911466,0.17385906593975609,-0.9282046384865223,-1.7929809124448939,-2.328636776283364,-2.4765803064566168,-2.21721804369158,-1.5732264381658414,-0.6109006851940366,0.5608111213768452,1.7949866594156485,2.917364337528333,3.745562760553157,4.119007994667209,3.939537525191551,3.20578035555236,1.9990126284722205,0.4440267413489343,-1.3231968477269893,-3.1771229122116917,-5.014504098519572,-6.758701335648006,-8.359003679809714,-9.787130980745115,-11.032547218916012,-12.09765581224364,-12.993489720176054,-13.73617619609674,-14.344239036888343],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=8","opacity":1.0,"showlegend":true,"visible":false,"x":[5.0411465101047845],"y":[8.353438586358578],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=9","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.14057140027303994,0.13779745226525306,0.13394958665715567,0.12875386869528815,0.12188055708770378,0.11294280637673919,0.10151172627208697,0.08717429285343327,0.06969537747152607,0.04941381731652445,0.028077875122818143,0.010018639905387074,0.0010799011751257339,1.461023140568827e-06,3.938663948735982e-25,1.824e-320,5.103269968946312e-14,1.09794504426673e-05,0.0009774289352789192,0.005572545047319623,0.012811663668895416,0.019640774003510966,0.024054633788159533,0.02530571018127387,0.023439502541747315,0.019013694270409588,0.013031669420115957,0.006940352120025311,0.002392738093842317,0.00034754164901230385,6.5544696450423385e-06,2.653698870018376e-10,1.7389607944820187e-26,3.3637362315297265e-136,1.9170174956089534e-308,2.1543128149450002e-163,7.083475480745266e-48,3.8960315423158356e-19,8.413894836327964e-14,0.0001807590842291199,0.18418313013499454,0.4792249577501559,0.19743594053581043,0.00023752901322700732,8.238590173674814e-25,4.598522789451187e-79,1.938902660200601e-232,3.575330419809842e-243,0.0,0.0],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=9","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[25.259279637305617,24.2681398955971,23.118464028451026,21.802684282482122,20.319254682963336,18.675025873461315,16.887824747587743,14.989001780088284,13.025563857912767,11.061336790680393,9.17641797061728,7.464024816426813,6.023786908829072,4.950673566473523,4.31923262351829,4.163786293447482,4.457720120933229,5.108981231019396,5.983172272290577,6.931036097174712,7.810910445883848,8.50534819462837,8.932144909477003,9.050404322515584,8.86227104515902,8.410729183003497,7.773568477696699,7.053417225647296,6.363803289167308,5.811715285562368,5.478272799007309,5.400962112097046,5.563228180124054,5.899027916899841,6.318518553035727,6.751329795548127,7.168274448742352,7.545511658824005,7.8640576477575825,8.119158692218882,8.290639073596079,8.35396148584117,8.298654724510103,8.107304650335665,7.7591753513505175,7.214682387509611,6.409812943794079,5.327816941531594,4.025772663152058,2.6254831684652546,2.617738078973122,4.00699498456494,5.311196762498126,6.402647254495368,7.207288472872945,7.752332361193847,8.096960851721862,8.288804707628882,8.34658966562037,8.279001222379737,8.110196250024686,7.855081274174915,7.528541272154949,7.154740007501571,6.744368657424349,6.3126195368593105,5.8792991310013685,5.416888356029184,4.862647479487956,4.164818040654351,3.312107942227911,2.341055428180844,1.327667643598471,0.3711213658660828,-0.4245406220638239,-0.9671490046362785,-1.1898857773858427,-1.0609570766533993,-0.5894406236811682,0.17264268835616825,1.1321820576524786,2.1612008492325123,3.1081318329227074,3.814833031691772,4.139657093627043,3.9858481343990495,3.325456268954638,2.1942997792450023,0.6701390905295477,-1.1479563269855015,-3.1559356202026096,-5.256212699744948,-7.364608106503526,-9.413634828089926,-11.35311009605179,-13.149009757447564,-14.78132108551292,-16.241461704932835,-17.529655301656096,-18.65250714228062],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=9","opacity":1.0,"showlegend":true,"visible":false,"x":[5.026932150430952],"y":[8.353817966113354],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=10","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.14154904985138048,0.1387805059353558,0.13491701613960883,0.12968129797321126,0.12273972915011931,0.11370117817242796,0.10213272109122597,0.08761951436868819,0.06993173615673892,0.049431358541478565,0.027925149965326294,0.009839116734003443,0.0010251919035459474,1.2084589100990427e-06,4.9854972494630023e-26,0.0,1.8523331101717738e-14,8.011076712266022e-06,0.0008064988933324129,0.004789533037761709,0.011172987120841619,0.01715872423734896,0.020865915582596532,0.02161293759300807,0.019509752890095024,0.015196671980499855,0.009761917955149019,0.0046616624089942955,0.0013154684464908653,0.00012568862710041575,8.150162093848098e-07,7.717128103609637e-13,1.592749164451241e-39,0.0,0.0,0.0,1.6168730012706332e-112,1.4676031889494119e-33,6.037740554410893e-54,6.894643974407606e-11,0.09624154143378379,0.5077161454343713,0.06882114071635284,1.478039220273286e-09,9.38931942743258e-204,4.456855037948351e-208,0.0,0.0,0.0,0.0],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=10","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[26.01604592381915,24.931670154468566,23.685050425374754,22.27088960136458,20.6904234110325,18.953691970488812,17.081927940727645,15.109812461865243,13.087217578411053,11.079901525238697,9.168470417223904,7.4448042854301795,6.0051297924255,4.9390987310061405,4.314718639941113,4.159897681198105,4.443641056615254,5.072686567949753,5.913835602330264,6.820407763673872,7.6540572004652345,8.301094222784316,8.683398472838565,8.764378471237976,8.5504619642585,8.088434478233165,7.458730745441389,6.764678348163916,6.117855537812858,5.620322142159742,5.345656794922937,5.322513320513885,5.526490976760355,5.8874125030153515,6.316888356463582,6.750541045079652,7.1650624360686725,7.53986006963215,7.858583968129062,8.117462983162792,8.292452851195844,8.35435006880882,8.29110322842802,8.091031665659767,7.744444377362302,7.214781784171525,6.426424003565192,5.3507111618180785,4.039904947946278,2.6116927830252403,2.610903852194673,4.027920671093662,5.342866753003391,6.425649615478315,7.212032799507926,7.743641562282208,8.087049736455814,8.28731553422435,8.35387935191834,8.287666131675245,8.114719136355259,7.856512316944115,7.530577519016996,7.159458829672482,6.749736569429556,6.3150887702727365,5.881522933057022,5.430164690028635,4.902161599192713,4.243510759516059,3.436459058569074,2.509647580682308,1.5320898581065139,0.5982548529613156,-0.18981114306208235,-0.7395004805038039,-0.9817766524695966,-0.8815928993398185,-0.44437524718827115,0.2814814655107165,1.2062575107090292,2.2050576017762102,3.129001507744449,3.82196945340087,4.143058740415807,3.9939713209994703,3.342176551335903,2.217006373887889,0.6897208392268768,-1.145796759043133,-3.1886976740436825,-5.342483902797718,-7.522188223753985,-9.658098361264791,-11.696872716857126,-13.60087205068788,-15.346410157401827,-16.921469352515082,-18.32327088455077,-19.555955416217756],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=10","opacity":1.0,"showlegend":true,"visible":false,"x":[5.022322927162973],"y":[8.353582242638396],"type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,0.98],"matches":"x2","showticklabels":false,"range":[-1,7],"title":{"text":"X"}},"yaxis":{"anchor":"x","domain":[0.575,1.0]},"xaxis2":{"anchor":"y2","domain":[0.0,0.98],"range":[-1,7]},"yaxis2":{"anchor":"x2","domain":[0.0,0.425]},"annotations":[{"font":{"size":16},"showarrow":false,"text":"Surrogate","textangle":90,"x":0.98,"xanchor":"left","xref":"paper","y":0.7875,"yanchor":"middle","yref":"paper"},{"font":{"size":16},"showarrow":false,"text":"Acquisition","textangle":90,"x":0.98,"xanchor":"left","xref":"paper","y":0.2125,"yanchor":"middle","yref":"paper"}],"title":{"text":"Gold Search"},"sliders":[{"active":0,"steps":[{"args":[{"visible":[true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"0"}],"label":"i = 0","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"4"}],"label":"i = 1","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"7"}],"label":"i = 2","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false],"title":"10"}],"label":"i = 3","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false],"title":"13"}],"label":"i = 4","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,false,false,true,false,false,true,true,true,true,false,false,false,false,false,false],"title":"16"}],"label":"i = 5","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,false,false,true,false,false,true,false,false,true,true,true,true,false,false,false],"title":"19"}],"label":"i = 6","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,false,false,true,false,false,true,false,false,true,false,false,true,true,true,true],"title":"22"}],"label":"i = 7","method":"update"}]}]},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Expected Improvement (EI)
Choose the next point that has the highest expected improvement over the current max.

$$
\begin{align*}
x_{t+1} &= \arg\max_x \mathbb E[\max\{0, h_{t+1}(x) - f(x^+)\}] \\
&= (\mu_t(x) - f(x^+))\Phi(Z) + \sigma_t(x)\phi(Z) \\ 
Z &= \frac{\mu_t(x) - f(x^+)}{\sigma_t(x)}
\end{align*}
$$

In [13]:
generation_strategy = create_generation_strategy(
    botorch_acqf_class=ExpectedImprovement,
)
ax_client = create_client(generation_strategy=generation_strategy)
predictions, acquisitions = run_ax(n=10, ax_client=ax_client)
results["EI"] = ax_client.get_trials_data_frame()

[WARNING 08-10 23:19:23] ax.service.ax_client: Random seed set to 1. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 08-10 23:19:23] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x', parameter_type=FLOAT, range=[0.0, 6.0])], parameter_constraints=[]).
[WARNING 08-10 23:19:24] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


In [14]:
render_plotly_html(
    plot_surrogate_and_acquisition(
        x=x,
        y=y,
        ax_client=ax_client,
        gold_fn=gold,
        predictions=predictions,
        acquisitions=acquisitions,
        line_kwargs=get_line_kwargs(),
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.34.0.min.js"></script>                <div id="c8894a68-fd63-40d8-9fa4-093f5634cf51" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("c8894a68-fd63-40d8-9fa4-093f5634cf51")) {                    Plotly.newPlot(                        "c8894a68-fd63-40d8-9fa4-093f5634cf51",                        [{"hoverinfo":"skip","mode":"lines","name":"Ground Truth (f)","showlegend":true,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[2.85,3.856123347536194,4.638417508253971,5.221510704359092,5.628898102467025,5.8829418136029314,6.00487089320169,6.01478134110787,5.9316361015757515,5.773265063269314,5.5563650592622364,5.296499867037907,5.008100208489416,4.7044637499195545,4.397755102040818,4.099005819975396,3.818114403255205,3.5638462958218278,3.343833886026604,3.1645765066304996,3.0314404348042556,2.948658892128277,2.919332044592691,2.9454270025973184,3.0277778209516644,3.166085498874992,3.3589179799961926,3.6037101523539157,3.896763848396514,4.233247844982003,4.607197863378097,5.011516569262348,5.4379735727217735,5.877205428253326,6.318715634763429,6.7508746355685645,7.1609198183944756,7.534955515376893,7.857953003061169,8.113750502402425,8.28505317876541,8.35343314192454,8.29932944606415,8.10204808977798,7.739762016069648,7.18951111235242,6.427202210449309,5.42760908659298,4.164372461425842,2.6100000000000705],"type":"scatter","xaxis":"x","yaxis":"y"},{"marker":{"size":10},"mode":"markers","name":"i=1","opacity":1.0,"showlegend":true,"visible":true,"x":[1.8140525221824646],"y":[4.153277647970732],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=2","opacity":1.0,"showlegend":true,"visible":true,"x":[5.636016067117453],"y":[6.4030358079544385],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=3","opacity":1.0,"showlegend":true,"visible":true,"x":[4.097510201856494],"y":[6.082020053347733],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=4","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.020387547294865707,0.015366768246353711,0.011042261234807756,0.007475513868284752,0.004692700060572605,0.0026729556257204003,0.0013407389506664192,0.0005679654410395972,0.000191652296832379,4.752959392155687e-05,7.808256260199064e-06,7.616091964912769e-07,4.255818513706739e-08,1.753443162904469e-09,1.2453468890936949e-10,5.4997700724870385e-11,2.537398675255238e-10,4.887853162516092e-09,1.131736462003843e-07,1.6558174847095385e-06,1.3513995542381337e-05,6.66243450772157e-05,0.0002228079363727343,0.0005581265389956207,0.0011277791080561098,0.0019389466244205227,0.002947960178436996,0.004080876173158875,0.005269082796720898,0.006493205253905086,0.007834111184501695,0.009536017602725417,0.01208680247897304,0.01628527096024659,0.023143965337439544,0.03339436352522444,0.04680085581598082,0.06201065612262954,0.07711786374848693,0.09035715099549944,0.10049069960765333,0.1069162555707797,0.10966655075737554,0.10940319013884739,0.10740742906764404,0.10546089214024584,0.10543677634159627,0.10860927306421563,0.11518377666651884,0.12451909201655227],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=4","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[5.879828741772096,5.723684944424338,5.564919034717484,5.405287615415519,5.246875209974377,5.092092146564724,4.943655173439945,4.804546034740408,4.677943045572257,4.567120898175266,4.4753146866562545,4.405545664614189,4.360408768005911,4.341825638785639,4.350771871227595,4.386993354512391,4.448748389221656,4.532810844813912,4.634893013277516,4.750143492373074,4.873588620047045,5.000504620600418,5.12671443494951,5.248806696645397,5.364276715061822,5.471590800773641,5.570176158374724,5.660339316717575,5.74311715203231,5.820066490320642,5.893001544577461,5.96369348171465,6.03355350613943,6.103329968324751,6.172860705100469,6.24095696127736,6.305593589211606,6.364345953453986,6.4148589511706895,6.45527159344706,6.484555617589533,6.502739129116825,6.51099724395559,6.511601248774515,6.50772643883592,6.503127171113509,6.501696264117567,6.50693847950454,6.521501446900843,6.547022552377477,6.101743942952618,6.170066285289676,6.219591383202151,6.250977680328067,6.2659653602492105,6.267250979972175,6.257994311081833,6.241256977277889,6.219539143177324,6.194444308974291,6.166489252021625,6.135068025432309,6.098571020386865,6.054651956833938,6.000626656609064,5.933977068904017,5.852922089199586,5.75696588950724,5.647101205867781,5.525598774378168,5.395669412032708,5.2610911299200485,5.125838350762745,4.993740002771914,4.868184535225848,4.751883097110591,4.646697269002982,4.553534562066633,4.472312966795951,4.401994655091324,4.340688005214141,4.285815939106026,4.234346690290113,4.183080205450835,4.128979247612083,4.069527932593617,4.003040362354105,3.928733988138434,3.8466290758021366,3.757397508226621,3.662195051125944,3.5624967627493516,3.4599483111916736,3.356240325414143,3.2530086498943227,3.1517603858862326,3.053823678413531,2.960318116876881,2.872142141022751,2.789973795802151],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=4","opacity":1.0,"showlegend":true,"visible":false,"x":[6.0],"y":[2.6100000000000705],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=5","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.09547155548148073,0.09012315367766023,0.08453948709746592,0.0787720664398825,0.0728881073413244,0.06697131832618429,0.06112171615641685,0.055454102773951235,0.05009491192355703,0.04517736368719812,0.04083529291677101,0.037196584830153166,0.034377586739854055,0.03247961967305596,0.03158713259346367,0.031763954144298466,0.033041616130494215,0.03540611640048351,0.038794872713660465,0.043093464203219016,0.04813429950004522,0.05370241453782443,0.05954889924604401,0.06540920317770327,0.07102273976886013,0.07615099584494331,0.08059256879159914,0.08419441901129032,0.08685889241837394,0.08854583946417775,0.08926872452821076,0.0890834100755615,0.08806885906321629,0.08630084524699297,0.08382299663181395,0.08062595053736829,0.07665853287522406,0.07186878513802215,0.06624980256540197,0.059877598921278995,0.05293279281664019,0.04570038845225446,0.03854433060017129,0.031857471444034065,0.025995162876749578,0.0212112005623313,0.017621453620349697,0.015212767362008174,0.013892070981472354,0.013547350311460243],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=5","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[8.468340204355687,8.319943402601783,8.162103186435765,7.995976068123989,7.823209376135191,7.645993524155708,7.467096662943996,7.289871876069473,7.118225242618438,6.9565316542639515,6.809484676401886,6.681867568229242,6.578235548940744,6.502505318316022,6.45745742886477,6.444170668648828,6.46145175268706,6.50569541071989,6.571495854392652,6.652408258291769,6.741622037602594,6.832523214153313,6.919139358810739,6.996468946138988,7.060701564171072,7.109336896755312,7.141209674344048,7.156425913745337,7.15621380429822,7.142691623430785,7.118556170748796,7.08669948415421,7.049770027996924,7.009707752544808,6.967300495386683,6.921885338350014,6.871562949633711,6.813846969790884,6.746346899355913,6.667371194662391,6.576404009663911,6.474425803939747,6.364060807091803,6.249544059739751,6.1365090223222944,6.031605268918414,5.941965861532494,5.874557492559809,5.835499974628444,5.8294539833374905,3.2724665946977454,3.4182967608792794,3.538823744584498,3.6342758575024776,3.706475543289276,3.7587991609158076,3.7955038902761644,3.820904419579552,3.838698211626939,3.8514758746081603,3.8604331269947023,3.865292100258253,3.864432497103712,3.855226161379832,3.8345602474380462,3.799522864928596,3.7482098110946405,3.680520197123244,3.5983718296197003,3.5052437617572867,3.40558711057715,3.304251191484301,3.205964912551628,3.1148964345447103,3.0343019525139363,2.966266870760491,2.911539022681024,2.869452968801623,2.8379457197751607,2.813666396287744,2.7921842861264805,2.768300427512023,2.736466194054345,2.6913074394014487,2.6282439610554027,2.5441780047790177,2.438045091795591,2.310720828619293,2.164546499486759,2.0028420939693357,1.8294693544129985,1.6484675080219744,1.4637701617226218,1.2790017289428839,1.0973454043894808,0.9214713725059718,0.7535128138303064,0.5950776265320101,0.44728501457652126,0.3108177663392855],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=5","opacity":1.0,"showlegend":true,"visible":false,"x":[0.0],"y":[2.85],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=6","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.0016156939514402062,0.0015020946828143617,0.001476951285408289,0.0015261094636664536,0.0016397834433463313,0.0018107938998608054,0.0020334092156318842,0.002303322538474272,0.0026188714192503016,0.0029832847700624002,0.003407562652365493,0.0039135711677260755,0.004536976148070978,0.005329536910577491,0.00635976697582091,0.007709885013455654,0.009465899789841742,0.011701760416568964,0.014464087284184355,0.017758950990433092,0.021543474585033837,0.025724982399300517,0.030167932034265366,0.03470668872617749,0.03916124552260405,0.04335319684833069,0.047120052266985284,0.05032675521508602,0.05287373880842449,0.054700971316898314,0.05578737362876173,0.05614501115902364,0.0558079218180883,0.05481663709245247,0.053201399240772815,0.05097072897762422,0.048119125606985856,0.044652202438302514,0.04061391979588341,0.03610716607905464,0.031301628871729506,0.026425029072346187,0.02173672612183378,0.017487234207159478,0.013873454503077113,0.011004926227529549,0.008895683454675363,0.007485619177898257,0.006680022156266285,0.006387424430193112],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=6","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[4.905908659808474,4.883643161404619,4.8817389891275385,4.897101983347304,4.926080698629625,4.964983663622537,5.010500342949675,5.060015090275039,5.111809104127176,5.165149156251933,5.220264192760564,5.278212672128369,5.340645688643097,5.409474511093833,5.486456957594952,5.572725440745373,5.668305399876559,5.7718853385236,5.881050737228021,5.992660909113121,6.103220658104033,6.209216974127871,6.307403930892759,6.395027181827009,6.469984984075921,6.530925828558005,6.577284111170234,6.609255552088237,6.627713995373938,6.634071544604074,6.630085453354766,6.617618507777067,6.59836543433898,6.573566510789392,6.543741045470279,6.508518397475282,6.466784135930759,6.417092020056604,6.35810101126255,6.2889653171373,6.209645467959559,6.12111938534537,6.0254810188553,5.925920932299997,5.826589143454272,5.732346580283711,5.648418488932788,5.579971472396698,5.531651619914407,5.507115466729512,3.442968390844645,3.575525456808435,3.689188863350435,3.7836872140412074,3.859898985688348,3.919862675682658,3.9663067737429736,4.002064471623041,4.029594985543918,4.0506358588043945,4.065996658066315,4.0754987491742884,4.0780609904946346,4.071926476035696,4.055019913829902,4.025417950477458,3.9819051798817346,3.9245255488821797,3.854728334922411,3.775045490646841,3.6886833805398527,3.599129406635324,3.509799451582297,3.4237416470447597,3.343404355451953,3.2704713050949312,3.2057641982972576,3.149212220722484,3.0998880203102543,3.056110171285135,3.0156121602578816,2.9757767993941773,2.9339319883429793,2.887698323073371,2.8353708808680693,2.776304656452756,2.711157675138022,2.6416782506286536,2.5702433614054576,2.499403308588491,2.431480639973862,2.36824565351417,2.3106821478871553,2.2588510232153025,2.2118554301783346,2.167908596765405,2.124503240251161,2.078678595305866,2.027376678928827,1.9678729118472071],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=6","opacity":1.0,"showlegend":true,"visible":false,"x":[3.7965335580594815],"y":[5.013611148163598],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=7","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.0005283532403099045,0.00048299732584098065,0.00047125507956924476,0.00048730946707455845,0.0005277517246496629,0.0005905449762644241,0.000674274524185547,0.0007778637374438102,0.0009008234783435334,0.0010439913820867843,0.0012106179349094532,0.0014076033647843517,0.0016466799977679716,0.001945303257500277,0.0023268469818069834,0.002819295208452804,0.003451145900209033,0.004244812394913112,0.005210147085247399,0.006338775323406302,0.0076006716962570935,0.00894461408564047,0.010303099043397207,0.011601059469603882,0.012766823914898515,0.013743377397508331,0.014497999082504057,0.015028539808348765,0.015364834336469076,0.015564053828300721,0.015699335578787083,0.015841908328514447,0.016038376051869203,0.016290452352754708,0.016548676485245637,0.016718804285812197,0.01668206989363889,0.01632630335384538,0.01557737673103757,0.014422142354260889,0.012916879089619247,0.011178958832088205,0.009363384564685606,0.00762987117652969,0.006109811919036718,0.004883932636214662,0.003978119547455646,0.003376780177627304,0.0030450755938535245,0.002948992543031849],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=7","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[4.5684466272314745,4.555117899206872,4.558076057018069,4.575026254384327,4.603236177709414,4.639906790496799,4.682473126829002,4.728827867715111,4.777464444987706,4.827539237558198,4.8788544004703684,4.931764564397158,4.987012712529612,5.045503618672188,5.108027824436347,5.174955458111631,5.245936821124397,5.319784003208837,5.394666234315999,5.468399404338625,5.538727979538544,5.603576119127385,5.661253388454609,5.71060601614968,5.751108506363404,5.782892881891758,5.806714422726894,5.823854109574371,5.835959752319554,5.844830629655051,5.852154943233817,5.85921582035776,5.866592988950883,5.873970947870852,5.880199131129726,5.883476510875628,5.881624396551974,5.87242570814218,5.853954462358419,5.824860492940233,5.784589452701239,5.733525500837681,5.67304984087915,5.605512825679806,5.534121343446785,5.462747345417814,5.395668195273985,5.337254517526978,5.291611471784249,5.262162167396656,3.6752712817560966,3.794507787232929,3.9014258348418602,3.995766218514203,4.078111058231951,4.149907386245252,4.213102147990471,4.269682342182964,4.3212952880716315,4.36896727578085,4.4129286717067435,4.452549826255212,4.486389049159393,4.512350986133621,4.527950279325532,4.530670798636689,4.51840456358776,4.489916288910697,4.445096789634493,4.384969270340802,4.311591058515114,4.227769074318627,4.136704931827135,4.041661581683777,3.945674608787188,3.8513211151289317,3.7605532079839796,3.674599065409801,3.593932055835686,3.5183069029221006,3.446860770111778,3.3782757300021795,3.3109967560201357,3.2434956094994334,3.174565509885193,3.103622930141228,3.0309149097952846,2.957413568760927,2.884529084178107,2.8138137340869056,2.7466933213004134,2.6842446145848937,2.6270309455640777,2.575003291088608,2.527470831910983,2.4831425914095626,2.440239677340136,2.396675212674867,2.3502956682164733,2.2991726367335303],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=7","opacity":1.0,"showlegend":true,"visible":false,"x":[4.331948131471222],"y":[6.909021517750929],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=8","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[8.344396423199845e-05,7.280309651927916e-05,7.228901458842519e-05,7.987374442755324e-05,9.494004757271294e-05,0.00011727246892313517,0.00014634869665920477,0.00018103799392710988,0.00021983191725560912,0.00026158004831789566,0.0003065134670039248,0.00035729270368977983,0.00041997510279754197,0.0005050421424017779,0.000628705828327605,0.0008142773295054784,0.001092157298118806,0.001496656065615034,0.0020593268636154736,0.0027987120493541055,0.003709320121554739,0.004754563109760982,0.005867270123772528,0.006958423687205954,0.007931975359726976,0.008702280208751962,0.00921075503219505,0.009439009230228073,0.009416249168163538,0.009219072637932136,0.00896218788333195,0.008779433604994575,0.008795285852675198,0.00909206931027279,0.009681028536642619,0.010471220031857149,0.011258498861707994,0.01176594513654034,0.011734880565110531,0.011025456550979975,0.009674378278763891,0.007885072029883543,0.005957524160472964,0.004186942761272318,0.0027728874724081507,0.0017787299960555902,0.0011555266060384534,0.0008043628401386724,0.0006326781488221691,0.0005800606428175846],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=8","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[4.506974582505489,4.490607525140319,4.499777607631226,4.529729818855024,4.574939363619504,4.6298913637683246,4.689708389661365,4.7506110584096195,4.810206338087978,4.867603782957409,4.923362876572384,4.979276719945801,5.038000426605721,5.102538718146161,5.175618018798251,5.258984751196508,5.352715050518732,5.454932809759227,5.5622388873186805,5.6703566991414025,5.774759574966053,5.871225338226678,5.956286791513584,6.027561413070375,6.083952839192781,6.125721675753908,6.154425435922397,6.1727287149732355,6.184086877929664,6.1923112308869745,6.201032235613269,6.213090438779924,6.229908964051747,6.25109085108975,6.274537553471567,6.296748091467201,6.313254640982461,6.319198355211747,6.3099260149541845,6.281588927814267,6.231691229435598,6.1595192380292705,6.0664025788824505,5.955778053224276,5.833045208008271,5.705219284200483,5.58040407833769,5.467124044360185,5.373554670206818,5.306661994851159,3.7038207846869584,3.8955943949798555,4.071609731333412,4.23161035911567,4.376592833949399,4.508796699905358,4.630972260899175,4.745496378082245,4.853685888011787,4.955356957311088,5.048653351522534,5.130152798574965,5.195247166147337,5.2387777620513685,5.255890739605391,5.243014202844184,5.198511300140733,5.122856160401623,5.018528542509786,4.889696834948538,4.741879111035671,4.5814476791780905,4.415048502952732,4.249044766248199,4.0890319918602245,3.939453504642943,3.8033322120292805,3.6821263085525686,3.57571197989455,3.482494132366016,3.3996449338828207,3.3234677370382437,3.249878993190717,3.174991474419773,3.095767445505481,3.0106871519644693,2.920199037110465,2.8264600302086507,2.7326664385349777,2.6423696039003444,2.5588608401365134,2.484666530745739,2.421177512864878,2.3684256504288212,2.3250138644479956,2.2882021565791524,2.2541493682522273,2.2183065785652554,2.175951385324516,2.122841613505034],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=8","opacity":1.0,"showlegend":true,"visible":false,"x":[4.587056987370298],"y":[7.691138525684522],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=9","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.0,8.698554203045079e-25,1.623619298661053e-08,3.655415036244262e-05,0.0007046955338102291,0.003005991896860502,0.0065407876240353895,0.00969221077948311,0.010882066813407244,0.009417186418237307,0.005894494368677824,0.0021651861047840954,0.00024565739729731466,5.890444744258549e-07,2.9901248973241963e-22,9.91328598577342e-226,2.1653566175059905e-12,1.3468101319539288e-05,0.0006941140142214571,0.003476593360402428,0.007529211436883851,0.010714506406092408,0.011603543008693479,0.010001852327719295,0.006755981906155952,0.0032972931229989203,0.0009509971211740053,9.80189147223561e-05,8.090059936560982e-07,1.1803642099358102e-12,8.303559344017288e-42,0.0,1.3708807580926702e-126,1.1169586570300156e-120,5.977425123651932e-101,5.6820728993879995e-50,1.5909709765010299e-15,0.00010921592658045284,0.10247444295293474,0.2752087219296989,0.4196868660646716,0.5129513839109431,0.5286142743028674,0.44038459479225517,0.23103365048522684,0.005299205137663439,2.6316369953722032e-76,8.880101699643955e-70,1.3908658457249528e-145,0.0],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=9","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[2.8551290177121422,3.136857747883641,3.8301290586562646,4.781943004254803,5.81592010343646,6.76621609564118,7.498488325149411,7.921011607436423,7.989542501614188,7.708730821369984,7.131440646509445,6.355686898921646,5.517299762600579,4.775224553476526,4.2860964155886,4.166252450727258,4.445573373622288,5.049536268453369,5.836977314670087,6.64644503479056,7.330527417817886,7.777474696172249,7.922653982933598,7.7528637996871375,7.305613462643527,6.663959239336917,5.946007544549145,5.287343420590473,4.815200954566043,4.61620746925414,4.705979346203553,5.018141541766383,5.436621704133843,5.878508410941112,6.32512021778686,6.7556717259710135,7.158856723183993,7.5329347052310345,7.894711835274651,8.284481427145373,8.681312258582166,8.981867957727015,9.056315462922418,8.818853312824963,8.26335574674558,7.445634916713615,6.428497202522309,5.242181162891001,3.92707004557826,2.623781362340391,2.6141327161210017,3.882771940864619,5.201473652208859,6.418701573855823,7.343961386462325,7.907734705367256,8.179380141121229,8.281364411206729,8.302879414319307,8.261394415477818,8.124122677178057,7.867644283090045,7.523475659570601,7.147698960713569,6.747161205676787,6.316600837678588,5.866063456360887,5.418293487557877,5.008527354313857,4.5991261486107815,4.097091302660224,3.4402473041133947,2.6371508588065105,1.7612820417918935,0.9238988055616368,0.24372526945824635,-0.17782165966104158,-0.2737305351841073,-0.021780802941202992,0.5502092102818308,1.3635174262196945,2.2927530700880423,3.1774303951983365,3.8440721191473384,4.141594466906811,3.989566698136069,3.417877134666237,2.5476172456508737,1.5425726661816226,0.5697295209075421,-0.22697290945702964,-0.7417147648633451,-0.9142200977466053,-0.7327461335970935,-0.23461757747445944,0.49532080686787827,1.3296164384870353,2.1091926919613964,2.663883633573373,2.8454534238731015],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=9","opacity":1.0,"showlegend":true,"visible":false,"x":[5.103586633859458],"y":[8.331261799710944],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=10","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.0,4.6006620049523106e-35,2.2850887388941583e-11,1.2349357972364261e-06,7.240599441974159e-05,0.0005089483652568077,0.00141105615023162,0.0023145890231480444,0.002598063435801204,0.002029017300308247,0.0009931170279281988,0.00021804511519206966,7.565918502783963e-06,4.704157918442892e-10,4.270645369248049e-36,0.0,1.266623058808536e-19,3.699346188138687e-08,2.1230048413631965e-05,0.0002586339413349799,0.0008236527424160322,0.0013609151610708729,0.0014713980180198788,0.0011081524009816162,0.0005574016060328355,0.00015860568235703937,1.693974149928131e-05,2.355318240658295e-07,1.5847571264854433e-11,3.4622866769315605e-24,1.0247307393775345e-94,0.0,0.0,0.0,0.0,5.486808318163093e-256,6.724060148422914e-117,1.843628158197414e-64,7.410564789538639e-15,0.000570854046040476,0.01053658310134061,0.015383865117828523,0.009552364606789949,0.0049345930285780186,4.302953822845676e-05,2.949985119638589e-22,0.0,1.6305352544978261e-220,0.0,0.0],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=10","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[2.8524657468214603,3.12952033035516,3.757487176596706,4.604870467574438,5.521091893815866,6.362093385282682,7.008816238643921,7.379225189895277,7.435011302520133,7.183958553324761,6.678446497786515,6.009915732776687,5.298559944697798,4.677264541062574,4.269173288255161,4.1595494749750275,4.366127748088958,4.829385316832487,5.437902532093004,6.061404143109643,6.5784655486374035,6.8967229701921795,6.965403719498402,6.780734438760591,6.384809611482695,5.858216753838953,5.30652014102836,4.8410041608975956,4.555272514776986,4.501699075009389,4.675221572468226,5.01521586679053,5.435155390544143,5.875888914233714,6.320681202997724,6.754938461808704,7.167489400625382,7.539128079462647,7.851489979449614,8.098202219023278,8.261471551657463,8.333937030974086,8.324996622747447,8.206027234545104,7.890263802808923,7.301502210301452,6.424947905444637,5.295779187195145,3.984102650760243,2.6185082814889697,2.613751054394086,3.9617065736934363,5.277057705326164,6.420187361837482,7.275183654394494,7.838340161485196,8.16927013210076,8.317671998537657,8.319779427866935,8.22371675443919,8.06621280332897,7.841824015390671,7.53450316274306,7.162188259400966,6.750612785223062,6.316337894038885,5.869366826978794,5.424786352592191,5.010460948499867,4.609496671867591,4.160761330946,3.6129137213276064,2.9593948127526937,2.2442286470426875,1.5485167610122077,0.9686942008990309,0.5942900634365995,0.48938199574733465,0.6794927747831143,1.1444537630572724,1.817444166819254,2.5906033583859593,3.327903196089125,3.885956954264269,4.142725231001319,4.03212172152882,3.5708285747766717,2.8476426586692103,1.991920245115633,1.1440847346678336,0.4323256181530808,-0.04398810093927841,-0.22490617844648764,-0.09556862394520227,0.3122501855266169,0.9225003566715886,1.6215462718829794,2.270129776833955,2.721695658901528,2.84769580253708],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=10","opacity":1.0,"showlegend":true,"visible":false,"x":[4.993654014104051],"y":[8.348254087751911],"type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,0.98],"matches":"x2","showticklabels":false,"range":[-1,7],"title":{"text":"X"}},"yaxis":{"anchor":"x","domain":[0.575,1.0]},"xaxis2":{"anchor":"y2","domain":[0.0,0.98],"range":[-1,7]},"yaxis2":{"anchor":"x2","domain":[0.0,0.425]},"annotations":[{"font":{"size":16},"showarrow":false,"text":"Surrogate","textangle":90,"x":0.98,"xanchor":"left","xref":"paper","y":0.7875,"yanchor":"middle","yref":"paper"},{"font":{"size":16},"showarrow":false,"text":"Acquisition","textangle":90,"x":0.98,"xanchor":"left","xref":"paper","y":0.2125,"yanchor":"middle","yref":"paper"}],"title":{"text":"Gold Search"},"sliders":[{"active":0,"steps":[{"args":[{"visible":[true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"0"}],"label":"i = 0","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"4"}],"label":"i = 1","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"7"}],"label":"i = 2","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false],"title":"10"}],"label":"i = 3","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false],"title":"13"}],"label":"i = 4","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,false,false,true,false,false,true,true,true,true,false,false,false,false,false,false],"title":"16"}],"label":"i = 5","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,false,false,true,false,false,true,false,false,true,true,true,true,false,false,false],"title":"19"}],"label":"i = 6","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,false,false,true,false,false,true,false,false,true,false,false,true,true,true,true],"title":"22"}],"label":"i = 7","method":"update"}]}]},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Upper Confidence Bound (UCB)

Choose the next point by trading off between surrogate's posterior mean and uncertainty.

$$
\begin{align*}
x_{t+1} = \arg\max_x \mu_t(x) + \sqrt{\beta} \sigma_t(x)
\end{align*}
$$

In [15]:
generation_strategy = create_generation_strategy(
    botorch_acqf_class=UpperConfidenceBound
)
ax_client = create_client(generation_strategy=generation_strategy)
predictions, acquisitions = run_ax(n=10, ax_client=ax_client)
results["UCB"] = ax_client.get_trials_data_frame()

[WARNING 08-10 23:19:24] ax.service.ax_client: Random seed set to 1. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 08-10 23:19:24] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x', parameter_type=FLOAT, range=[0.0, 6.0])], parameter_constraints=[]).
[WARNING 08-10 23:19:25] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


In [16]:
render_plotly_html(
    plot_surrogate_and_acquisition(
        x=x,
        y=y,
        ax_client=ax_client,
        gold_fn=gold,
        predictions=predictions,
        acquisitions=acquisitions,
        line_kwargs=get_line_kwargs(),
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.34.0.min.js"></script>                <div id="362df455-0893-40a8-bda0-bdc3a3804f31" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("362df455-0893-40a8-bda0-bdc3a3804f31")) {                    Plotly.newPlot(                        "362df455-0893-40a8-bda0-bdc3a3804f31",                        [{"hoverinfo":"skip","mode":"lines","name":"Ground Truth (f)","showlegend":true,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[2.85,3.856123347536194,4.638417508253971,5.221510704359092,5.628898102467025,5.8829418136029314,6.00487089320169,6.01478134110787,5.9316361015757515,5.773265063269314,5.5563650592622364,5.296499867037907,5.008100208489416,4.7044637499195545,4.397755102040818,4.099005819975396,3.818114403255205,3.5638462958218278,3.343833886026604,3.1645765066304996,3.0314404348042556,2.948658892128277,2.919332044592691,2.9454270025973184,3.0277778209516644,3.166085498874992,3.3589179799961926,3.6037101523539157,3.896763848396514,4.233247844982003,4.607197863378097,5.011516569262348,5.4379735727217735,5.877205428253326,6.318715634763429,6.7508746355685645,7.1609198183944756,7.534955515376893,7.857953003061169,8.113750502402425,8.28505317876541,8.35343314192454,8.29932944606415,8.10204808977798,7.739762016069648,7.18951111235242,6.427202210449309,5.42760908659298,4.164372461425842,2.6100000000000705],"type":"scatter","xaxis":"x","yaxis":"y"},{"marker":{"size":10},"mode":"markers","name":"i=1","opacity":1.0,"showlegend":true,"visible":true,"x":[1.8140525221824646],"y":[4.153277647970732],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=2","opacity":1.0,"showlegend":true,"visible":true,"x":[5.636016067117453],"y":[6.4030358079544385],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=3","opacity":1.0,"showlegend":true,"visible":true,"x":[4.097510201856494],"y":[6.082020053347733],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=4","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.5385537455963063,-0.586919132471347,-0.6353555829071795,-0.6834275172359378,-0.7306248079001517,-0.7763541519346802,-0.8199287658387051,-0.8605552153030349,-0.897315243803269,-0.9291389589480386,-0.9547638702576812,-0.9726741465483395,-0.9810253962753392,-0.9776108885815831,-0.9600583667768401,-0.9265613037683571,-0.8770132383870983,-0.8133804260585877,-0.7387206493432023,-0.6560692544643613,-0.5679836547461643,-0.4765359212254989,-0.3834248644437226,-0.2900741243979332,-0.19769333173470782,-0.10730902652184487,-0.019774021337617576,0.06423938655426303,0.14425954945891076,0.22004624022986016,0.2916244349646481,0.3592985766156801,0.42359611088465926,0.4850656492096811,0.5439126580066164,0.5996628156545168,0.6512023207783356,0.6971815842352724,0.7364206221845143,0.7681279177031611,0.791968029180581,0.8080734775267988,0.8170640008159974,0.8200909417511887,0.8188691337476174,0.8155700173953773,0.8123905876617088,0.8108293232923862,0.8112170583316103,0.8129835691100369],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=4","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[5.879828741772096,5.723684944424338,5.564919034717484,5.405287615415519,5.246875209974377,5.092092146564724,4.943655173439945,4.804546034740408,4.677943045572257,4.567120898175266,4.4753146866562545,4.405545664614189,4.360408768005911,4.341825638785639,4.350771871227595,4.386993354512391,4.448748389221656,4.532810844813912,4.634893013277516,4.750143492373074,4.873588620047045,5.000504620600418,5.12671443494951,5.248806696645397,5.364276715061822,5.471590800773641,5.570176158374724,5.660339316717575,5.74311715203231,5.820066490320642,5.893001544577461,5.96369348171465,6.03355350613943,6.103329968324751,6.172860705100469,6.24095696127736,6.305593589211606,6.364345953453986,6.4148589511706895,6.45527159344706,6.484555617589533,6.502739129116825,6.51099724395559,6.511601248774515,6.50772643883592,6.503127171113509,6.501696264117567,6.50693847950454,6.521501446900843,6.547022552377477,6.101743942952618,6.170066285289676,6.219591383202151,6.250977680328067,6.2659653602492105,6.267250979972175,6.257994311081833,6.241256977277889,6.219539143177324,6.194444308974291,6.166489252021625,6.135068025432309,6.098571020386865,6.054651956833938,6.000626656609064,5.933977068904017,5.852922089199586,5.75696588950724,5.647101205867781,5.525598774378168,5.395669412032708,5.2610911299200485,5.125838350762745,4.993740002771914,4.868184535225848,4.751883097110591,4.646697269002982,4.553534562066633,4.472312966795951,4.401994655091324,4.340688005214141,4.285815939106026,4.234346690290113,4.183080205450835,4.128979247612083,4.069527932593617,4.003040362354105,3.928733988138434,3.8466290758021366,3.757397508226621,3.662195051125944,3.5624967627493516,3.4599483111916736,3.356240325414143,3.2530086498943227,3.1517603858862326,3.053823678413531,2.960318116876881,2.872142141022751,2.789973795802151],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=4","opacity":1.0,"showlegend":true,"visible":false,"x":[5.28214746108094],"y":[8.062511138469858],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=5","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.3645544599693048,-0.40620889697438295,-0.4487693572292339,-0.49192446251275784,-0.5352867931952072,-0.5783809667447924,-0.6206302365267578,-0.6613417172991769,-0.6996908783446095,-0.734707209821059,-0.7652656554245167,-0.790093446698328,-0.8078097463659468,-0.8170232570448337,-0.8165106445340357,-0.805470278608273,-0.7837843463918496,-0.7521019716974495,-0.7116312674951332,-0.6638439501296887,-0.610234563847381,-0.5521738522199584,-0.4908427898560223,-0.42721627164676834,-0.3620706508665546,-0.29599995394036943,-0.22943391205926533,-0.16265600006365405,-0.09582268167457597,-0.028987258501393776,0.03786622588228028,0.10477179492431296,0.17170559767877988,0.2384816417742628,0.30462628455505275,0.3692708242365497,0.43118429861835716,0.48894474321941783,0.5411367947973023,0.5865383405029793,0.6242835810080976,0.6539958781882218,0.6758757202486636,0.6907123257587958,0.6997878473159334,0.7046478498566683,0.7067206022031154,0.7069921589392384,0.7059138541771491,0.7035878155004716],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=5","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[8.089770783263793,7.860703606414598,7.622117300893126,7.376099688338648,7.1253219785175155,6.873069748980079,6.623248354985064,6.380352899230695,6.149391768108006,5.935752228225059,5.744997078470032,5.58258337413377,5.453498361348393,5.361814567530566,5.310175925435652,5.299239875807018,5.327143502321956,5.38943477851384,5.479778491746931,5.59080935235496,5.714888049790666,5.84473582507414,5.973937716933844,6.0973122090921255,6.211149692918072,6.313324411622921,6.4032851704564395,6.4819300423332695,6.5513705965315605,6.614592876266389,6.675026473408285,6.73604051585881,6.800396840039718,6.86970610537781,6.943951357454329,7.02123206846183,7.098170925213172,7.170833401164502,7.235622256157267,7.289998308777529,7.332970781549623,7.365330749429897,7.38962938420428,7.409928660757309,7.431374597500959,7.459632097178986,7.500162634007157,7.557375773358315,7.633914594137186,7.730588830641307,6.057821125822733,6.230772420779854,6.371089301576157,6.476336139352336,6.545461224194559,6.5788961681584315,6.57849542239355,6.547538323694363,6.490510558127817,6.412296036017706,6.317348203152218,6.209111962187286,6.089735703082352,5.960086505855439,5.820057379900498,5.669128591613826,5.507115907422042,5.334960267867072,5.155100335004108,4.971280644884314,4.788145989892904,4.610759434375058,4.444109019682951,4.2926493069475855,4.159908160836719,4.048177439531344,3.9582984796420484,3.8895487960916713,3.839634325606318,3.804790792464836,3.779997306493481,3.7593040739037846,3.7362731330785035,3.704525497667028,3.6583795253964198,3.593550986385698,3.507730674810319,3.4005898048328476,3.2734146272854217,3.128687205120473,2.96967844483979,2.800084369444627,2.62372251482942,2.444294529204493,2.265213518386304,2.0894899480624605,1.9196673836692617,1.7577984227955428,1.6054513354061761,1.4637387576621972],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=5","opacity":1.0,"showlegend":true,"visible":false,"x":[5.715464231391885],"y":[5.7787487222264815],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=6","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.4134886197966897,-0.45455734654670155,-0.4968745571657266,-0.5401341389664015,-0.5839413156600692,-0.6277971917928746,-0.6710813125660736,-0.7130323705440398,-0.7527278176722061,-0.7890646165350805,-0.8207465138281738,-0.8462891956131811,-0.864064098433035,-0.8724115757131474,-0.8698529598639331,-0.8553993566887226,-0.8288810588370025,-0.7910574138038355,-0.7433578973910886,-0.6875235595871645,-0.6253226531389444,-0.5583809485831153,-0.48810694968801366,-0.41567444135838544,-0.3420322054535073,-0.2679233950974983,-0.19390662722791885,-0.12037664009079979,-0.04758592027585479,0.02432848576723784,0.09530589138793927,0.16530080996971824,0.2341891420803035,0.3016361380649871,0.36693535502046004,0.42887065774623934,0.48575744727800374,0.5356591951261093,0.5766346679482592,0.6069706907311291,0.6253949414097821,0.6312802383864801,0.6248503638174534,0.6073709745403796,0.5812554232376044,0.5498735910367925,0.5167823601568517,0.48477712556070046,0.45534888090941783,0.4288731299785815],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=6","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[7.576300754476621,7.399635932766348,7.212976444204291,7.017743821689516,6.815917149801075,6.610092318577717,6.403521627786201,6.200122507013764,6.004441928939392,5.821561341955104,5.65692614511161,5.516084567874959,5.404324170014147,5.326201019632968,5.28496783320163,5.281923380666701,5.315757910066951,5.382409802064481,5.475806973348677,5.588773054151802,5.71381771963976,5.8437874049713905,5.972370993502309,6.094464072793638,6.206399984830421,6.306056958048309,6.392849286957459,6.467608136148658,6.532355443601211,6.589973965377437,6.64377915101628,6.697005657735562,6.752234098102101,6.810802656912812,6.872273114200918,6.934126560972997,6.9921956865557755,7.041671805235057,7.0780864956820935,7.098109589986862,7.100111119604184,7.08446953324168,7.053639856567529,7.01202396419479,6.965707464856334,6.922033247018113,6.88875903062333,6.872863262483258,6.879288417619,6.9104013765599435,5.822273132560526,5.9912315208525335,6.131473200357386,6.240537467006419,6.317258108938576,6.361776532783678,6.375464578991521,6.3610345630503895,6.32246307670857,6.264189492797135,6.190245233462045,6.1036398985259215,6.006050309329502,5.897830956053315,5.7783394126480125,5.646541724892816,5.50183021782603,5.344893706028416,5.1781085967511755,5.00528892320304,4.831217004282355,4.661114692425046,4.50012438754282,4.352844083976038,4.222941523406904,4.112859521836505,4.023617129870907,3.9547082672096545,3.904099440399525,3.8683296805528062,3.842717522904155,3.821680313818009,3.79916902737602,3.7692158792300603,3.726581493635981,3.6674700149148265,3.5901078610879296,3.494695390876932,3.3829653801445367,3.2577041046273383,3.122308549513858,2.980410819165079,2.835583172753023,2.6911245397058647,2.549921416427718,2.41437178001636,2.2863590659595414,2.1672634466752116,2.0579988983109385,1.959066323815064],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=6","opacity":1.0,"showlegend":true,"visible":false,"x":[5.0165540567741385],"y":[8.353042361423036],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=7","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.36274088699771156,-0.3881799851477882,-0.4184122481783814,-0.45422181423564867,-0.49645855090730473,-0.5460132397436911,-0.6037764910343892,-0.6705765353418018,-0.7470905966398264,-0.8337244806212194,-0.9304554217959884,-1.0366335043447155,-1.1507316383837898,-1.2699792663108902,-1.3891004041680317,-1.454132961531623,-1.353286213510054,-1.2415720782684567,-1.1338134954298575,-1.033908370931564,-0.9440423343477122,-0.8653142950192151,-0.7979970859129981,-0.7417107382123834,-0.6955344459615816,-0.6580597598771463,-0.6273865917172258,-0.601065928089213,-0.5759966676744728,-0.5482889678365598,-0.5131119229335825,-0.46453606231792366,-0.395233462386914,-0.29320312928481157,-0.08414411934622121,0.19984122837017676,0.47523647866274965,0.7245636440531938,0.935018864971134,1.0958779325158234,1.199065308195765,1.2424110037996832,1.2148695291645935,1.0737436330425894,0.8121193129701074,0.43121331247149314,-0.045370321918906026,-0.5257505253715541,-0.9316041302614675,-1.2511332278229312],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=7","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[12.56345373133549,12.35626091083674,12.093520159625392,11.764123007086429,11.356736043566272,10.86096607367824,10.269131261165466,9.578728728478108,8.795586756999981,7.937489832493709,7.037734057286789,6.147593199980982,5.336087492708497,4.684890050715658,4.275996588296173,4.170453723850872,4.380007698977817,4.855326550110301,5.509292197208063,6.243762579055863,6.967345472614708,7.604862399730535,8.100828866082617,8.419479978921196,8.543425584797543,8.472342389867212,8.222408316783635,7.826514337123495,7.33463696214579,6.813106297987871,6.340920150204285,6.000955423372265,5.864391133742632,5.968660468586983,6.293701019272335,6.752450228523464,7.227830689813137,7.630573040959817,7.928507849296807,8.138147399184959,8.28677634751282,8.367861745803182,8.326563745509945,8.106692260901758,7.695648880240936,7.102626938994457,6.371377777239635,5.613624812565279,4.983736200654257,4.600415600136827,3.940189100944672,4.775688640562266,5.579887195518154,6.357633468155488,7.072585924044096,7.6677365529465416,8.087014175299975,8.3076607340964,8.346495331798138,8.209825254091875,7.912985551767977,7.516836663468899,7.109909392365481,6.762382259103688,6.487531361672997,6.232682892297781,5.907184283416737,5.438145789736037,4.808357097308292,4.050988145136694,3.2291859776210066,2.416199017527731,1.6811690496537266,1.081174039026556,0.6580222555193802,0.43777758154640534,0.4312532808141243,0.6342649682915309,1.0270579771825696,1.5729566517207059,2.2169348419995436,2.8854897586387382,3.489854990942269,3.9350036077496533,4.136649144780818,4.046764110383214,3.6756499421015296,3.0807503165414047,2.3395987403161214,1.529372396419225,0.7150116839175418,-0.05566324187737681,-0.7521209875874355,-1.358676035663219,-1.8710020733806534,-2.2925956490458814,-2.6317169583276376,-2.899010475823361,-3.105812581667477,-3.2630537702004165],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=7","opacity":1.0,"showlegend":true,"visible":false,"x":[5.045794885225497],"y":[8.352948194884533],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=8","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.6360814102529371,-0.6525555475115393,-0.6729752157076666,-0.6981418854263717,-0.7289552720837588,-0.766399125890082,-0.8115134558287226,-0.8653486162066143,-0.9288959318384364,-1.0029889360190651,-1.0881690576420027,-1.1845100419743242,-1.2913968808936265,-1.4072571498729758,-1.5292318880899427,-1.6077938448838565,-1.49058046523736,-1.3790684696619881,-1.2770090789025101,-1.1864436950365274,-1.1082833346326577,-1.0425627335838519,-0.9886363752461993,-0.9453172794618773,-0.9109586605428512,-0.8834814220250699,-0.8603530740722093,-0.8385265807439963,-0.8143519432742058,-0.7834804048151537,-0.7407929409582855,-0.6804037473411751,-0.5958168409613929,-0.4802747530096896,-0.24964146805436765,0.052410802234749754,0.33372487914019877,0.5794751109485696,0.7775980699032167,0.9194499434201546,1.0002499013587423,1.019717289036653,0.9889724474923199,0.8618736154502842,0.6519947297174815,0.307953477842337,-0.19541907217864654,-0.797587158782355,-1.3834070141344537,-1.8878524812419435],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=8","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[14.899487098845947,14.59731824625456,14.219299303029757,13.752573302449534,13.184908973420045,12.50629333222417,11.711155560461776,10.801245388207903,9.789054259773252,8.701440274025298,7.582779828056482,6.496525298267478,5.523557081900988,4.755332914857103,4.279850256945082,4.159288377387758,4.401687111531412,4.949738428555891,5.704339242251746,6.552252234819074,7.385596009375586,8.113398437382568,8.667095180999294,9.002207970879372,9.098139325732944,8.957422472041884,8.605068993439598,8.08795015561714,7.4734659139563195,6.846145746387824,6.300416096830881,5.927840723195359,5.798178219339359,5.93625889353408,6.30148458813715,6.787046172700685,7.267471551058512,7.659951544891383,7.946252560064422,8.150426982512341,8.29220366903258,8.353690466491,8.296126679967244,8.101448379150634,7.75771517139026,7.210870616330006,6.423709561752764,5.46159286000988,4.4992454043535,3.7173729849391735,3.4363744798695195,4.448386089366773,5.459534061882181,6.4231928485077265,7.2065143041822495,7.753768893192425,8.100878037864577,8.293719029346061,8.353364606224746,8.269792707957093,8.03352553758157,7.673142112342545,7.262181253093345,6.879546555345489,6.558681308408256,6.261994347928036,5.89942624182173,5.386156493646027,4.690416252141956,3.837041100903333,2.8905934909793167,1.9346816090369696,1.0546214210789047,0.325696758424332,-0.193456140144602,-0.46390309769677973,-0.46757853228987933,-0.20785561994798218,0.28977780687320775,0.9769024424246795,1.7818699546774024,2.611372446976623,3.355817515431063,3.9007390033472946,4.146114545225724,4.033807505948925,3.570915495805598,2.81858575469758,1.863813383412925,0.7971554325291041,-0.30127334576935905,-1.3686421198749938,-2.3610573960803016,-3.2520187545426635,-4.0292842902159745,-4.69126371249193,-5.243614544617011,-5.696379529279881,-6.0617765403625725,-6.352617664133875],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=8","opacity":1.0,"showlegend":true,"visible":false,"x":[4.9993789174230505],"y":[8.349843198387447],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=9","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.796464324431568,-0.8121960086675974,-0.8318440902939169,-0.8561997518668545,-0.8861462728038825,-0.9226443155428149,-0.9667045364759566,-1.0193434292555343,-1.081517637643299,-1.154031414063758,-1.2374116104070108,-1.331744801912472,-1.4364720753084201,-1.5501380295854703,-1.6700777482577924,-1.7476382208868741,-1.6328984019018842,-1.5245444345197972,-1.4259975718809406,-1.3390442774742342,-1.2644037136188453,-1.2019612849805137,-1.1509443161970534,-1.1100438363122145,-1.0774842182643856,-1.0510445513735878,-1.0280379753978766,-1.0052580983039348,-0.9789059504970099,-0.9445179227124105,-0.8969264908469698,-0.8303034812669818,-0.7383606065495942,-0.6147336095956321,-0.3810172689731264,-0.0808528632588304,0.19842830063232972,0.44275086612839026,0.6409165875123386,0.7851381746784056,0.8711503538410656,0.898399340492169,0.8673460751281438,0.7404519246779215,0.5276293663742984,0.18085468117389628,-0.32472969151527276,-0.9314305584665407,-1.527875864420319,-2.0494747045627135],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=9","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[15.131163680845056,14.793425971981248,14.376228559970052,13.867659639665115,13.25695982290163,12.536144483045446,11.702174733262288,10.759658820866607,9.723933856755961,8.62416960777616,7.505842242835078,6.43155966647697,5.4788401499898,4.73318763685476,4.274911568255967,4.158949202708486,4.389962841629986,4.912797605312843,5.6334829821189345,6.44411969641588,7.2409310446980815,7.935510370006517,8.460715911296433,8.77302086885251,8.852923219512984,8.704533141916055,8.354858349973975,7.852688601300532,7.2663925750092595,6.679451080333329,6.182298089301333,5.859284910115759,5.770730128871854,5.932556859478343,6.300215400515167,6.771989855237504,7.23476122306415,7.618238152631982,7.910024196300097,8.129948381049752,8.286473817404215,8.353497617725283,8.296166782154828,8.101615778557067,7.756560007201858,7.20918082543622,6.4233917716398725,5.461225268407107,4.489480636557436,3.6809727051443843,3.4229645130906388,4.442762796454438,5.459252603272109,6.422885242441785,7.205320189774224,7.753156049852195,8.101082792601348,8.294393836153073,8.353328471826304,8.275020540858932,8.054811417725952,7.712554109384889,7.309440788545064,6.919055952180956,6.57981430303293,6.266106114804881,5.9000988282887565,5.402474429925359,4.738372081616255,3.924727437626209,3.0176111380568953,2.094204230553497,1.2364111556806119,0.5188374579889987,0.0012595765124698843,-0.275478503905064,-0.29148427898111073,-0.048866168092808415,0.4275430549277548,1.0897202038644433,1.8667839055917224,2.667256810226082,3.384871390796116,3.9097329495178834,4.146294635499876,4.038178248384177,3.5891183747524043,2.853363950977284,1.9107244119071365,0.8463501805152678,-0.26248123937455947,-1.353316955618995,-2.380771498571095,-3.315718380548933,-4.142829545361105,-4.8574817873661384,-5.462682841663506,-5.966375905939984,-6.379273074715096,-6.713236228633146],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=9","opacity":1.0,"showlegend":true,"visible":false,"x":[5.038983862056284],"y":[8.353604872575431],"type":"scatter"},{"hoverinfo":"skip","mode":"lines","name":"a=10","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.9202538824104529,-0.9361281076901904,-0.9559634328765323,-0.9805597387881465,-1.010809715070308,-1.0476839337978432,-1.092203141538557,-1.145393652830414,-1.208221053755085,-1.2814968588364621,-1.3657524694714844,-1.4610749891882888,-1.5669003712907343,-1.6817603835111794,-1.8029672159873853,-1.8813452868152323,-1.7653206911216284,-1.6557916751126793,-1.5561933687014187,-1.4683172131062157,-1.3928775108758797,-1.3297463276751715,-1.278130026265389,-1.2366914502324664,-1.2036195881149088,-1.1766506881862933,-1.1530471690628563,-1.129543608504193,-1.102273492532014,-1.0666975071812943,-1.017565672478554,-0.948963818813985,-0.8545202948424676,-0.7278015013410749,-0.49004459421484614,-0.18548331741287727,0.09799910025020273,0.3462436492169475,0.5480190764337439,0.6955399691521506,0.7845139501525253,0.8132552198705765,0.7813180391613506,0.6526007906376129,0.4356672010744566,0.08260531918789378,-0.43211500836373296,-1.0502361148624217,-1.6585833566111916,-2.191261666771573],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=10","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[15.010767877655201,14.675115076385527,14.26077834775662,13.756045613569139,13.150371176904452,12.435979307628633,11.610005334987935,10.677154208962277,9.652726469043914,8.565656947610911,7.460923553017993,6.400325535506802,5.460260747183061,4.724882263033541,4.273122131022453,4.158868138751297,4.386534341062323,4.901720795965945,5.611839318521209,6.41056524233714,7.195601120113388,7.879774482007088,8.396882583662189,8.704048843428335,8.782158749002221,8.635467133252204,8.29088288555173,7.796829869582867,7.2210069212029335,6.645893775601972,6.160610996087088,5.847996638583442,5.766909544525106,5.932282199727337,6.299570121339346,6.768434030660749,7.2280525628632875,7.610213384891543,7.903331062386096,8.126312714444,8.285562899872508,8.353562555737353,8.296229247464236,8.101623100930786,7.75641050428671,7.209134351406138,6.423462285229208,5.460969322899211,4.487851176940622,3.675858028389271,3.4229308351597676,4.442169939339741,5.459057064726086,6.422973820877711,7.205401068959955,7.75314361753708,8.101115454012925,8.29475131622086,8.353435689998593,8.275181805060276,8.056803456565278,7.717114370233848,7.315334159003698,6.924044891637993,6.582307825555239,6.266402555053479,5.900628718889338,5.406950271345,4.750495231964813,3.9472388051799028,3.0518428498734016,2.1400312191145203,1.292447063293769,0.5827382684478182,0.07004402123401832,-0.20513638722885652,-0.22299918082243764,0.014491578873689903,0.4828808456903464,1.1347774053167536,1.9002133820688476,2.688914342108518,3.3960571340947894,3.913286379899034,4.146406740047306,4.0397759646283165,3.596880181245991,2.870933950499743,1.9403920150868181,0.8890790107696467,-0.20682366238564764,-1.28564996066528,-2.3024941994604404,-3.228449913894181,-4.048214149110743,-4.757058647279818,-5.357811282566192,-5.858202971961673,-6.268730563917068,-6.6010560913442085],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=10","opacity":1.0,"showlegend":true,"visible":false,"x":[5.035939505891532],"y":[8.353772660864934],"type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,0.98],"matches":"x2","showticklabels":false,"range":[-1,7],"title":{"text":"X"}},"yaxis":{"anchor":"x","domain":[0.575,1.0]},"xaxis2":{"anchor":"y2","domain":[0.0,0.98],"range":[-1,7]},"yaxis2":{"anchor":"x2","domain":[0.0,0.425]},"annotations":[{"font":{"size":16},"showarrow":false,"text":"Surrogate","textangle":90,"x":0.98,"xanchor":"left","xref":"paper","y":0.7875,"yanchor":"middle","yref":"paper"},{"font":{"size":16},"showarrow":false,"text":"Acquisition","textangle":90,"x":0.98,"xanchor":"left","xref":"paper","y":0.2125,"yanchor":"middle","yref":"paper"}],"title":{"text":"Gold Search"},"sliders":[{"active":0,"steps":[{"args":[{"visible":[true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"0"}],"label":"i = 0","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"4"}],"label":"i = 1","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"7"}],"label":"i = 2","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false],"title":"10"}],"label":"i = 3","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,false,false,true,true,true,true,false,false,false,false,false,false,false,false,false],"title":"13"}],"label":"i = 4","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,false,false,true,false,false,true,true,true,true,false,false,false,false,false,false],"title":"16"}],"label":"i = 5","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,false,false,true,false,false,true,false,false,true,true,true,true,false,false,false],"title":"19"}],"label":"i = 6","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,true,false,false,true,false,false,true,false,false,true,false,false,true,false,false,true,true,true,true],"title":"22"}],"label":"i = 7","method":"update"}]}]},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Thompson Sampling
Choose the next point by sampling from the surrogate's posterior (or a sum over a batch of samples) and then optimizing along this trajectory.

In [17]:
generation_strategy = create_generation_strategy(
    botorch_acqf_class=PathwiseThompsonSampling
)
ax_client = create_client(generation_strategy=generation_strategy)
predictions, acquisitions = run_ax(n=10, ax_client=ax_client, acqf_n=5)
results["Thompson"] = ax_client.get_trials_data_frame()

[WARNING 08-10 23:19:25] ax.service.ax_client: Random seed set to 1. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 08-10 23:19:25] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x', parameter_type=FLOAT, range=[0.0, 6.0])], parameter_constraints=[]).
[WARNING 08-10 23:19:27] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


In [18]:
render_plotly_html(
    plot_surrogate_and_acquisition(
        x=x,
        y=y,
        ax_client=ax_client,
        gold_fn=gold,
        predictions=predictions,
        acquisitions=acquisitions,
        line_kwargs=get_line_kwargs(color="red"),
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.34.0.min.js"></script>                <div id="bcc0b6cf-2763-4282-89a8-b3321e07c0ee" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("bcc0b6cf-2763-4282-89a8-b3321e07c0ee")) {                    Plotly.newPlot(                        "bcc0b6cf-2763-4282-89a8-b3321e07c0ee",                        [{"hoverinfo":"skip","mode":"lines","name":"Ground Truth (f)","showlegend":true,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[2.85,3.856123347536194,4.638417508253971,5.221510704359092,5.628898102467025,5.8829418136029314,6.00487089320169,6.01478134110787,5.9316361015757515,5.773265063269314,5.5563650592622364,5.296499867037907,5.008100208489416,4.7044637499195545,4.397755102040818,4.099005819975396,3.818114403255205,3.5638462958218278,3.343833886026604,3.1645765066304996,3.0314404348042556,2.948658892128277,2.919332044592691,2.9454270025973184,3.0277778209516644,3.166085498874992,3.3589179799961926,3.6037101523539157,3.896763848396514,4.233247844982003,4.607197863378097,5.011516569262348,5.4379735727217735,5.877205428253326,6.318715634763429,6.7508746355685645,7.1609198183944756,7.534955515376893,7.857953003061169,8.113750502402425,8.28505317876541,8.35343314192454,8.29932944606415,8.10204808977798,7.739762016069648,7.18951111235242,6.427202210449309,5.42760908659298,4.164372461425842,2.6100000000000705],"type":"scatter","xaxis":"x","yaxis":"y"},{"marker":{"size":10},"mode":"markers","name":"i=1","opacity":1.0,"showlegend":true,"visible":true,"x":[1.8140525221824646],"y":[4.153277647970732],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=2","opacity":1.0,"showlegend":true,"visible":true,"x":[5.636016067117453],"y":[6.4030358079544385],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=3","opacity":1.0,"showlegend":true,"visible":true,"x":[4.097510201856494],"y":[6.082020053347733],"type":"scatter"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=4","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-2.371453415081201,-2.2556421898639494,-2.135969612839455,-2.014072124215035,-1.8914827826639273,-1.7695907258917778,-1.6496077171179997,-1.5325430667570776,-1.4191877986314063,-1.3101084342599578,-1.2056502142794079,-1.105948983779466,-1.0109503621167637,-0.9204342233069106,-0.8340419566544246,-0.7513034844205926,-0.6716618172307005,-0.5945088276493662,-0.5192437617951389,-0.4453285644844063,-0.3723298300931306,-0.29994626363348,-0.2280212490646505,-0.15654080906206236,-0.08561806398421146,-0.015466228808261206,0.05363680657962622,0.12138821940067324,0.18749916514561482,0.2517266654651835,0.3138912847730091,0.37387247209328267,0.43157257617347433,0.48683973558269233,0.5393392987966403,0.5884023563578196,0.6330632347799412,0.6722679938021314,0.7050488685206591,0.7306424114362036,0.7485605685377332,0.7586242047640743,0.7609689207612316,0.756033413850583,0.7445410781039926,0.7274859701260011,0.7061346614411049,0.6820531456345472,0.6570969815031581,0.6332453790326844],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=4","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.29575836774608133,-0.32820503998071593,-0.3595637923500037,-0.3893088072125972,-0.4172831670724886,-0.4436791272315761,-0.4689921504647665,-0.4939516550005294,-0.5194331679115276,-0.5463581780789186,-0.5755893717261897,-0.6078300474690475,-0.6435373021020547,-0.6828590283085976,-0.7256048690401711,-0.7712610516554097,-0.8190526362834523,-0.8679771805336798,-0.9167426275439082,-0.9637233139822444,-1.0069710648883297,-1.044274668373409,-1.0732589144141231,-1.0915133075879773,-1.096739716931005,-1.086907686438712,-1.060405954407096,-1.016178951182812,-0.9538376685005594,-0.873735297575661,-0.7769993690085899,-0.665513722342068,-0.5418453926092348,-0.4091133144944175,-0.2707975981270504,-0.13050973547538008,0.008161639575193752,0.14172976375551027,0.266907684003618,0.3807266253014776,0.4806606727895384,0.5647443384471842,0.6316707360989667,0.6808596282369362,0.7124864740603343,0.7274657145743445,0.7273837709821908,0.7143806383276055,0.6910115581183622,0.6601468208610402],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=4","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.5358020501104861,0.4180189475832765,0.30509593091978227,0.19813435700857274,0.09731677994913457,0.001974584777145072,-0.089265507311038,-0.1782685802197872,-0.2671305017236085,-0.3578913649525408,-0.45224891319678995,-0.5512986453084634,-0.6553267104065597,-0.7636794372272497,-0.874729626327593,-0.9859548975062429,-1.094130356833361,-1.1955317070794762,-1.286051777486783,-1.3613605993266322,-1.4171387550284893,-1.4493570483751717,-1.4545732817101635,-1.4302174534754997,-1.3748388635536046,-1.288292313138556,-1.1718455683108786,-1.0281961793032874,-0.8613921930715503,-0.6766578041202437,-0.48013108671655685,-0.2785261939823781,-0.07873640500862078,0.11260315798594267,0.28957350575791185,0.4474073822753162,0.5827482918955802,0.6937253537978452,0.7799578237278375,0.8424961535383906,0.8836938186625638,0.9070108769181238,0.9167564927440522,0.9177830171428422,0.9151483275987395,0.913765760495866,0.9180619683470429,0.9316629093310044,0.957140109672311,0.9958571804247338],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=4","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-1.9306641979339894,-1.9359417015418523,-1.9326107340544312,-1.9194580594825277,-1.8953303753041684,-1.8592009922848802,-1.810243121214582,-1.747904099864348,-1.671974268190844,-1.5826440635782053,-1.4805432790725959,-1.366757282393552,-1.2428162675862333,-1.1106552040937563,-0.9725439309003306,-0.8309886688302706,-0.6886092470258254,-0.5480153251225657,-0.41170380304861764,-0.2819596870669767,-0.16075960562248867,-0.0496861408054996,0.050139646490230116,0.13810156781181498,0.21410341486943052,0.2785537461776002,0.33232039829853766,0.37665680939177215,0.41310404321940575,0.4433738142080881,0.4692187172655211,0.49229617265487224,0.514032259930207,0.5354906333423343,0.5572502006454608,0.579306962437411,0.6010770156609021,0.6214951850072621,0.6391360270012336,0.6523452297444685,0.6593782816346025,0.6585417094299387,0.6483311920426339,0.6275605310846408,0.5954758130769979,0.5518500974556909,0.4970555330004739,0.4321109586546703,0.35868234954363243,0.2789973063579855],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=4","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.6097852311049462,-0.6406313075386327,-0.680436865149814,-0.7277081884104063,-0.7806695170130692,-0.8373357904934786,-0.895594474926628,-0.9532916006778415,-1.0083169255028057,-1.0586832424546921,-1.1025952562455354,-1.1385041118763124,-1.1651445135282965,-1.1815523436449609,-1.1870616969085401,-1.1812811941602424,-1.1640517877514747,-1.1354094089756446,-1.0955736259534699,-1.044936089259855,-0.9840420826093373,-0.9135691036103553,-0.8343059681376254,-0.7471350990129121,-0.6530197245603708,-0.5529967495978895,-0.4481751497043962,-0.339738956739021,-0.2289533193268588,-0.11717179264146294,-0.005842975486570312,0.1034851137490953,0.20916395332591864,0.3094469053716602,0.40249421971042254,0.4863867324517859,0.559190913491499,0.619069477570416,0.6643922516649311,0.6938388447387556,0.7064906666835675,0.7019097895251973,0.6802025100626976,0.6420662832374127,0.5888199273143087,0.5224186085688325,0.4454570263339115,0.36116438327988587,0.2733449178923455,0.186177880837318],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=4","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[5.879828741772096,5.723684944424338,5.564919034717484,5.405287615415519,5.246875209974377,5.092092146564724,4.943655173439945,4.804546034740408,4.677943045572257,4.567120898175266,4.4753146866562545,4.405545664614189,4.360408768005911,4.341825638785639,4.350771871227595,4.386993354512391,4.448748389221656,4.532810844813912,4.634893013277516,4.750143492373074,4.873588620047045,5.000504620600418,5.12671443494951,5.248806696645397,5.364276715061822,5.471590800773641,5.570176158374724,5.660339316717575,5.74311715203231,5.820066490320642,5.893001544577461,5.96369348171465,6.03355350613943,6.103329968324751,6.172860705100469,6.24095696127736,6.305593589211606,6.364345953453986,6.4148589511706895,6.45527159344706,6.484555617589533,6.502739129116825,6.51099724395559,6.511601248774515,6.50772643883592,6.503127171113509,6.501696264117567,6.50693847950454,6.521501446900843,6.547022552377477,6.101743942952618,6.170066285289676,6.219591383202151,6.250977680328067,6.2659653602492105,6.267250979972175,6.257994311081833,6.241256977277889,6.219539143177324,6.194444308974291,6.166489252021625,6.135068025432309,6.098571020386865,6.054651956833938,6.000626656609064,5.933977068904017,5.852922089199586,5.75696588950724,5.647101205867781,5.525598774378168,5.395669412032708,5.2610911299200485,5.125838350762745,4.993740002771914,4.868184535225848,4.751883097110591,4.646697269002982,4.553534562066633,4.472312966795951,4.401994655091324,4.340688005214141,4.285815939106026,4.234346690290113,4.183080205450835,4.128979247612083,4.069527932593617,4.003040362354105,3.928733988138434,3.8466290758021366,3.757397508226621,3.662195051125944,3.5624967627493516,3.4599483111916736,3.356240325414143,3.2530086498943227,3.1517603858862326,3.053823678413531,2.960318116876881,2.872142141022751,2.789973795802151],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=4","opacity":1.0,"showlegend":true,"visible":false,"x":[3.5404561478325185],"y":[4.20264814790707],"type":"scatter"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=5","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.6408217594013437,-0.6030654315565309,-0.5629644494832444,-0.5240413827463721,-0.4895791870775294,-0.46219703170839704,-0.44355752430495043,-0.4342439364927302,-0.43381773551289027,-0.441037300170913,-0.454192820484711,-0.47149415617196466,-0.4914407518653714,-0.5131068046437202,-0.5362901236974202,-0.5614971253929258,-0.5897640811440041,-0.6223265292015286,-0.6601759755111457,-0.703600368797314,-0.7517936578974054,-0.8026039419615079,-0.8524719318995176,-0.8965851785737231,-0.929242550129881,-0.9443924765596509,-0.9362822758354834,-0.900138486184194,-0.8327922833462773,-0.7331707443230805,-0.6025902936494678,-0.444766326078063,-0.2654714624530021,-0.07194404463673032,0.12786649623285973,0.326120311193673,0.5157353245449198,0.6907986572713747,0.8468313905007092,0.9808877228892185,1.0914772087975448,1.1783295555355573,1.2420476519303314,1.2837121225545693,1.3045072031490155,1.3054323064023279,1.2871473495818426,1.2499756979718635,1.1940653794621716,1.1196855721524224],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=5","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.715549621630424,-0.6051490106276713,-0.487712652584864,-0.3766165929133779,-0.28259717769274534,-0.2124465488050139,-0.16835197449017653,-0.1479723485358061,-0.14522240107901185,-0.1516180283966751,-0.15794498106974764,-0.1559636675454366,-0.13986336759346923,-0.10722920098939914,-0.059375550328740145,-0.0010140783956105182,0.06066073443733494,0.11733075198528722,0.16114761938003666,0.1862210694190119,0.18973382380073472,0.17250845461975417,0.13894973354962437,0.09639452086104337,0.05400100839164475,0.021382574071013155,0.007226337168448949,0.018127829583767685,0.057824260201282784,0.12693023590534475,0.22318485876577104,0.34208597890732156,0.47770731304118913,0.6235827571962838,0.7735073159603086,0.922103895699135,1.0650416537441898,1.198907481839654,1.3208376035086853,1.4280436716777924,1.5173883495980023,1.5851547767714107,1.6271097260625957,1.6388907438425293,1.6166674463914825,1.5579531421137573,1.4623908893021338,1.3323212601580092,1.1729993770269989,0.9924184610094079],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=5","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-1.1342796695600323,-1.1425064789358932,-1.136010680685137,-1.1170712968841519,-1.0894012933770743,-1.0578545577360272,-1.0279997813818202,-1.0055988507340783,-0.9960372668393958,-1.0037595217705104,-1.0317626487974159,-1.0811961849740217,-1.151106877090184,-1.2383524193460786,-1.3376915139536618,-1.4420390536302898,-1.5428636549522432,-1.6307793757795865,-1.696359257403763,-1.730979350087814,-1.7275862637402977,-1.6813326001587612,-1.5900334131286469,-1.454409446238628,-1.278099217308697,-1.0674405759258776,-0.8310414772875692,-0.5791776587584727,-0.3230700448555126,-0.07410566071856328,0.15693125614849013,0.36060321295956177,0.529543809366753,0.6590352071994708,0.7473714261960651,0.7959782582123833,0.8091909177568954,0.7937235546012835,0.7579746592422051,0.7112359906143357,0.6628664762676464,0.6214961936598868,0.5943232688374206,0.5865585376856959,0.6010598685645043,0.6381813386016499,0.6958435080800579,0.7698118928072443,0.8541614115989575,0.9418964278406536],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=5","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.8656511127644682,-0.9770648551081574,-1.0763843670049653,-1.166190417398775,-1.2486968867942596,-1.3252384894226066,-1.395906191452432,-1.4593780573014452,-1.5129689034430533,-1.552894850500523,-1.5747217862677756,-1.5739430092320206,-1.5466136607873002,-1.4899600292194302,-1.4028815462217903,-1.286272364395259,-1.143110683699432,-0.9783398095279346,-0.7985862384648922,-0.6116683175604243,-0.4259246679032782,-0.24943959124038306,-0.08925547395040578,0.04933486670662979,0.16332970729601326,0.25232339408344984,0.3185274167732859,0.3664880402711752,0.4025103999630498,0.4338267092091478,0.46757882276032686,0.5098781239209415,0.5652604598806459,0.6363593743469091,0.7237590478998681,0.826057918201129,0.9399557768410413,1.0603894588907081,1.1809098210000277,1.294272446057609,1.393161496924597,1.470954849883815,1.5224371185368673,1.5443755074902943,1.5358906948352098,1.4985792188836158,1.436372443473553,1.3551471590185173,1.2621366448287858,1.165218460395162],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=5","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-1.6031478993771207,-1.6175875459224034,-1.6217747552990418,-1.6161277531584965,-1.6013245628715975,-1.5782668191286167,-1.5480277490577925,-1.5117868613977348,-1.4707551716129548,-1.426095993701729,-1.3788473799472591,-1.3298531305270416,-1.2797098838245475,-1.2287381135851516,-1.1769849021332948,-1.124266155875017,-1.0702502081188539,-1.0145129161188942,-0.95650215899234,-0.8955131007733306,-0.8307061730153495,-0.7611606721778503,-0.685955489463119,-0.6042681329642096,-0.5154831291382731,-0.4193011607719885,-0.31584094191391554,-0.20572684733112728,-0.09015667612498386,0.029054419991822816,0.14945856322852574,0.2680386305785985,0.38141202898787047,0.48610304440806745,0.5788512516023914,0.6569272159786101,0.7182830814167576,0.7615453231482756,0.7860038031394292,0.7916088846337082,0.7789655077889883,0.7493152008222896,0.7044993434340763,0.64689944564346,0.5793526537885527,0.505043036072748,0.42737132202532496,0.34980771944304434,0.27573775872533124,0.20831523659399975],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=5","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[6.597104567533039,6.458592465956468,6.314222885524656,6.165126356924852,6.012738132613691,5.85880863205245,5.7053991524775025,5.554857924983656,5.409771302781305,5.272885010344755,5.146991190907552,5.03477874387941,4.938647447305872,4.8604908938183895,4.80145949103243,4.761722553109143,4.740270250607933,4.734968918623298,4.742996987325901,4.761336419677615,4.787190981668237,4.818311564431616,4.8532174348142085,4.8913077535153215,4.932862111151966,4.978932643002691,5.031134121986967,5.091342907399503,5.161321137335792,5.242289004631531,5.334484845663015,5.436941568928736,5.547710496748168,5.664184235511376,5.783357920509535,5.902046181553647,6.017092339466086,6.125602178034202,6.225192855596025,6.314219779591802,6.391948738930194,6.458649100755947,6.51559192809538,6.564944229655722,6.609557575696781,6.652656577317007,6.697440691517194,6.746625156194639,6.802056093465325,6.8646360036415635,5.586672956572552,5.657310494191146,5.707753639101652,5.73772563819518,5.748109921169657,5.740936182589992,5.7189562716027895,5.685125488077387,5.642168205091851,5.592253653132436,5.53679456688849,5.476374306827757,5.410801679073093,5.339286552034765,5.2607229586927495,5.1740593423491115,5.078728064108786,4.975057598462273,4.864355508232654,4.748660985266467,4.630499818444913,4.512694983620957,4.398208024265235,4.2899772061290244,4.190714891741552,4.102680166644134,4.027462685448508,3.9658069729584624,3.917498757162885,3.8813283424691396,3.8551406152478638,3.8359765903585465,3.8203068224095875,3.804351867927563,3.78447882317689,3.7576540694137837,3.7218441655461882,3.676097986253575,3.62040339461513,3.555492633634322,3.4826370629107295,3.4034526025616127,3.319729058157094,3.233289932421146,3.1458845075681987,3.05911073812557,2.9743655400441247,2.892818087369027,2.8154014487914845,2.7428180763093124],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=5","opacity":1.0,"showlegend":true,"visible":false,"x":[5.915544456967698],"y":[3.7145500599253864],"type":"scatter"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=6","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.42422877170831214,-0.23069936319153506,-0.06709906288628942,0.06818601580122385,0.1807784270179461,0.2782722785920656,0.36779841304991445,0.4538415434744706,0.5368875571594331,0.6132547737178626,0.6761452105101753,0.7176252443977067,0.7309961696620859,0.7129100761777687,0.6646530705497788,0.5922384322274408,0.5052686487303792,0.41485788352991526,0.3311578079517694,0.26112739483065295,0.20713001632252648,0.16672631427518447,0.13371579910105336,0.10015010152164239,0.05878362171380269,0.005311003215274557,-0.060203917366763005,-0.13309105934295307,-0.20509993558959394,-0.26642953341927367,-0.3082406874720687,-0.32495399017012305,-0.315671748745864,-0.28442243072654616,-0.23919031782228295,-0.18997354547681145,-0.14632952519872555,-0.11505199078909278,-0.09862751878784026,-0.09486969825872693,-0.09781333498398803,-0.0996201415772493,-0.09298123780495171,-0.07337122402647345,-0.04054842880147391,0.0011041872162478317,0.043495869428338496,0.07676748051761884,0.091735099743526,0.0821724535164472],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=6","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-1.4614245762827291,-1.5204698779189019,-1.5723948339726022,-1.6140926225781653,-1.6429909022907352,-1.6571837939258596,-1.6555155956601737,-1.6376145618397202,-1.6038785785280045,-1.555417448600838,-1.4939585536215447,-1.4217237854656761,-1.3412858353110408,-1.2554112802558104,-1.1668965799478541,-1.0784012951675361,-0.9922838874433602,-0.910484860402875,-0.8344953210933896,-0.765352216001622,-0.7036403856088163,-0.6495032223830148,-0.6026638771395971,-0.5624588743100882,-0.5278859725231608,-0.4976679730419298,-0.4703337915775858,-0.4443173745973612,-0.418073931037549,-0.39021150985869557,-0.35963303859819207,-0.32565924128795753,-0.2880949469412428,-0.24726362307398309,-0.20401468834207748,-0.15970120364049833,-0.11617329021306899,-0.07577577318399764,-0.04129338971072194,-0.015837128786572396,-0.0026767731550876306,-0.005028134770972192,-0.02580647315553608,-0.06735979264938852,-0.1311967782671044,-0.21772377870979587,-0.3260033175142043,-0.45354835183020303,-0.5963008772074161,-0.7490504471151813],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=6","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.7455256579044872,-0.7312985534310505,-0.717097098731028,-0.7074061338445211,-0.7080016179337623,-0.7249251755681003,-0.763327591306441,-0.826377114544013,-0.9144210269226002,-1.0245502506687774,-1.1506531951671466,-1.2839670528301992,-1.4140552773811554,-1.5300722004805682,-1.6221309586830066,-1.6825766392828636,-1.7069794333817137,-1.6946324643336625,-1.6484191906530137,-1.5741785298527278,-1.4796986938495116,-1.3734917359654206,-1.2635376429581497,-1.1561927494195818,-1.0554306467637748,-0.9625291934407606,-0.8762438722983535,-0.793427805072383,-0.7099854715495503,-0.6219926764576337,-0.5267874133846167,-0.4238219162079522,-0.31509683748123174,-0.20511306951560387,-0.10033528780240003,-0.008252824832091832,0.06367505755833647,0.10902752472934249,0.12334535977756536,0.10481257322800577,0.05456359297204641,-0.023448942802273984,-0.12301307012019376,-0.23655581847366325,-0.35623479998141844,-0.475043425209616,-0.5877476779996857,-0.6915020564923342,-0.7860641486706954,-0.8736132094196893],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=6","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-1.7857052103977922,-1.6946435158249873,-1.5905573366013104,-1.4846865300417953,-1.3870894870145036,-1.3050964230438222,-1.2423084960144841,-1.198318683069273,-1.1691873185090822,-1.148556953092579,-1.1291673632216472,-1.1044567260432618,-1.069923844110395,-1.0239806356109808,-0.9681324862021691,-0.9064642052072058,-0.8445531618801333,-0.7880556696120132,-0.741279354918459,-0.7060418037147673,-0.6810627202689824,-0.662028951648459,-0.6423311752565379,-0.6143302897611653,-0.5708994818774373,-0.5069275753622458,-0.4204731444030676,-0.31332650255345346,-0.19085462249521112,-0.06114861565911811,0.06636536554623085,0.18259500845482501,0.2804208086945912,0.35596290107612116,0.4092187699387637,0.44394896717424603,0.4668395155189746,0.4861082000568584,0.5098283777652781,0.5443028289178429,0.5928098629845434,0.6549690920359538,0.7268507328733381,0.8018049005155675,0.8718458089551471,0.9293186170546993,0.9685254453361336,0.9869987820408896,0.9860990705631804,0.9706661002781094],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=6","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.7131149192640667,-0.6359981848213807,-0.567904017060442,-0.511993867961932,-0.4698279129639055,-0.44126408266732364,-0.4245864784909236,-0.41684500831440113,-0.4143566323689685,-0.41329417468947094,-0.41027494853359925,-0.4028605952629185,-0.3898917546830051,-0.3716047126884782,-0.3495085317708973,-0.3260356724885582,-0.3040146362307796,-0.28607992285699146,-0.2741443321295299,-0.2689649741419585,-0.2698619242296674,-0.27464650514558053,-0.2797840276062312,-0.28077952880631224,-0.2727404505448745,-0.2510424684947525,-0.2120080388549733,-0.15350429270146662,-0.07537831076817125,0.02032792429586261,0.12941085718627426,0.2458494398173297,0.36250515204303774,0.47199086470564605,0.5675639051927793,0.6439311743960499,0.6977892709734994,0.7280394763970328,0.7357274088320982,0.7237210553149946,0.6961672968873218,0.6578006091692061,0.6132005345547026,0.5661033322248961,0.5188665283875719,0.4721637538143709,0.4249542341913819,0.37473093271346447,0.31800132803390135,0.25091121714971465],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=6","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[6.184732534053777,6.117070841836798,6.04598470131922,5.971973254041631,5.89569963889174,5.8180019783243395,5.73989749480339,5.662576916791322,5.587385995409042,5.515790797748399,5.449323595595962,5.389506785482738,5.337753537345662,5.295245954500293,5.262794566957911,5.240686959570309,5.228546049905443,5.22532684668071,5.22953887639039,5.239506384475378,5.253591944426224,5.270373894937296,5.288772555356667,5.308122702465235,5.328191630401677,5.349143518786639,5.371452171604988,5.395765850605045,5.422730237636328,5.452778673795082,5.485907555350123,5.521547202830074,5.558646060860566,5.595811775753826,5.631436870420612,5.663818731055677,5.69129490067763,5.712395933871579,5.7259963715021405,5.731446006406218,5.728668225872712,5.718216153194623,5.701280596317893,5.679646576027022,5.655597794550709,5.631771222920401,5.610967356157333,5.595925247404899,5.589064079650803,5.592183698993074,4.401907018593774,4.46900985813848,4.525545676274649,4.5712330251915905,4.6064334583519315,4.632169690884214,4.649882304487544,4.661123097416409,4.667301522927197,4.669495565400596,4.668331316373074,4.663932869319203,4.655942051257701,4.643605647664299,4.625925770965301,4.601866548757276,4.570607181544452,4.531803684211383,4.485680808743269,4.432939905823349,4.3746621405438955,4.312236560183756,4.247285265115821,4.18153625992962,4.1166668756180504,4.054153019551975,3.995139789588581,3.940343895183274,3.889994869990743,3.843819424365655,3.8010713100353826,3.7606074183278797,3.721009109271561,3.680745592522708,3.6383732841210725,3.592760513698559,3.5432796732594922,3.4898270605644846,3.4327286749532986,3.3726275439534503,3.310372755559327,3.2469197764559126,3.183247346530214,3.120292940692842,3.058906486456977,2.9998205833024354,2.943634702063706,2.890810564753464,2.841675956812484,2.7964344764374323],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=6","opacity":1.0,"showlegend":true,"visible":false,"x":[2.745669727375028],"y":[2.923531982028244],"type":"scatter"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=7","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.2849788374554474,0.20476803790000675,0.12367024372367275,0.045048802794158904,-0.027581129874393318,-0.09095385613893314,-0.14243286955960865,-0.1803183180497363,-0.2040400869533694,-0.2142095563368902,-0.2125225122160188,-0.20152641370653046,-0.1842840712722651,-0.1639799593085527,-0.1435228866217586,-0.12519876673413988,-0.110418662735428,-0.09957370233943297,-0.09199551978202192,-0.08605422901483073,-0.07939327742364055,-0.06928228189972141,-0.05306839914095174,-0.02871251523182128,0.004682472055626885,0.046803148609345535,0.09644935063941273,0.15184430631925644,0.2108856557186483,0.27134369455724394,0.33101359187938545,0.38781418219100894,0.43982933760972986,0.48535383263359844,0.522996161064178,0.5518264090037437,0.5713047860369076,0.5810303859738042,0.5805922496052589,0.569544317001673,0.5474741322922752,0.5141292289072132,0.4695641870168963,0.41427644771275096,0.34931013793570465,0.27632330674830285,0.19763304289810524,0.11626350216839709,0.03580588390962003,-0.04028101363236258],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=7","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.5526957772446782,-0.42318799583004185,-0.2976109567946058,-0.17626869389463007,-0.059578812760097355,0.051876159266919464,0.15730335208442514,0.25566929399155036,0.34569386639379984,0.4258759172548967,0.49455492299122894,0.5500087079069409,0.5905817899983448,0.6148328040409528,0.6216831172745967,0.6105426889769949,0.5813917842348584,0.53489881055599,0.4726410479879981,0.3972536367902324,0.3124359876348526,0.22280458227543587,0.13358661238921554,0.05015438188644228,-0.02253006064981744,-0.0802207369158845,-0.11945723983488732,-0.13763830675297828,-0.13307679786914084,-0.10497438764785955,-0.05327046339610572,0.02142183761232236,0.1175009568601513,0.2320826560416153,0.3608979908723258,0.4983696498609642,0.6378971287895119,0.7723059254490612,0.8943556090433521,0.9972569500459475,1.0751593578391092,1.1235740547829849,1.1397051401068188,1.122669509797541,1.073596826048406,0.9956115735011004,0.8937098490986914,0.7745472650472374,0.6460081382308046,0.5163050688697313],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=7","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-2.784509908502123,-2.899318878823789,-2.9951603015227253,-3.0670763933648857,-3.110995971296904,-3.1240254859014636,-3.104654165906208,-3.0528556252057357,-2.970077622718514,-2.8591221330655894,-2.7239286405286878,-2.569283754589455,-2.400489075293284,-2.223026068042963,-2.0422611064975467,-1.8632356294436316,-1.6905713804160702,-1.5283265666030847,-1.3796536425937904,-1.2465379654776216,-1.1297041277844837,-1.0286669365153314,-0.9418954836856125,-0.8670539508134638,-0.8012919577622621,-0.7416062367411491,-0.6852316580937186,-0.6299819866304924,-0.5745166222184934,-0.5185257710927035,-0.4628305385150826,-0.40930119107391827,-0.36047573735285,-0.31904895217478013,-0.28731706782548194,-0.2666379967429815,-0.2572001242139157,-0.25809622762156526,-0.2674546615526523,-0.28260668253965693,-0.30029822051951194,-0.3169434861700797,-0.3289077550957309,-0.33279835030062777,-0.3257368418088331,-0.30558218366360546,-0.27107405586170974,-0.22187241501775645,-0.15859207524832208,-0.08303499051422025],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=7","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.6170207963390953,0.4302364983772704,0.24503925744386001,0.06671781660774923,-0.1000156231642817,-0.25125573357658715,-0.38406597651743635,-0.49657030443354144,-0.5879769637310912,-0.658539277201287,-0.7094592407083788,-0.7427392959212753,-0.760986029288452,-0.7671671493326645,-0.7643202444964934,-0.7552087789361722,-0.7419329160513686,-0.7257040873113697,-0.7069682907050907,-0.6855957421787969,-0.6610506528486226,-0.6325631426912024,-0.599326448871778,-0.5607415108881756,-0.516677805953329,-0.4675014560110456,-0.4139151521596257,-0.35684425328608,-0.29738946392062426,-0.236835958705574,-0.1767068845007717,-0.11876809058702686,-0.06487003501328387,-0.016746640511001304,0.024179161502872915,0.05696922651521347,0.08119164336477118,0.09680377839583226,0.10403035524573793,0.10325424878704971,0.09492044953984338,0.07945661529787855,0.05721615286998746,0.028451390719111375,-0.006675195815132956,-0.04803470375201006,-0.09542571214010746,-0.14842012641666424,-0.2062526279107701,-0.26792000249687964],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=7","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.5526209459690778,0.4926567561353177,0.41877349995685786,0.329036821013729,0.22316124923764574,0.10288431470987813,-0.027945092705484287,-0.16359514316637636,-0.29697852172709843,-0.4204139041560767,-0.5265180263217726,-0.6091014202326103,-0.6639361089423903,-0.6892784349774567,-0.6860626599664922,-0.6577262086059066,-0.6096826072231967,-0.5485609448538339,-0.48136535704959366,-0.4146115489866424,-0.3535549618260947,-0.30163985368519,-0.26026753136833336,-0.22893733049647047,-0.2057195124099832,-0.1877887037151169,-0.17195202797514808,-0.15523960575536955,-0.13543944056575863,-0.11144511228249965,-0.08332428700231975,-0.0522217817594085,-0.02030634473896309,0.00942859077253061,0.03352141855136104,0.04846520790542441,0.051297990040337194,0.040229324944058065,0.015002265662559633,-0.023068855147341916,-0.07136380878215387,-0.12645973680485567,-0.1847719176230162,-0.24321113205099532,-0.2997339267804752,-0.3536773067410438,-0.4058001068181435,-0.4580003754151544,-0.5128783075875905,-0.5734839187821521],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=7","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[6.508958126564162,6.408558844651116,6.299639203643293,6.182480511441747,6.05764607068696,5.926023274839881,5.788857500205727,5.647771826138925,5.504765524169902,5.362183581723069,5.222649664169403,5.088956401571277,4.96391036213817,4.850135229507676,4.74984606287787,4.664620048804745,4.595215467522117,4.54164539103693,4.503585668592251,4.4807564290363775,4.473128218036828,4.480927698455155,4.504446189252032,4.543680281036149,4.597937178996509,4.665863172313989,4.7458072002202805,4.836083676461141,4.935051565208114,5.041007339536925,5.151929189928936,5.265287745728499,5.378161278290685,5.4874084954815014,5.589779646680886,5.682002784024148,5.760923644972414,5.823739557521482,5.868283727901077,5.893301359150439,5.898673371788739,5.885557348672539,5.856426106125573,5.814992474234103,5.76601583996399,5.714993421202607,5.667748482576115,5.6299386026950815,5.606501391924936,5.601030190599446,4.284351235369802,4.386019278007182,4.468420844199696,4.531031467470775,4.574718745974307,4.601785680792973,4.615418117154045,4.618989241145032,4.615492928394624,4.607132459148087,4.595073155288467,4.579360668994705,4.559001633080164,4.532198872634271,4.4967278127686825,4.45043280430625,4.391811262020639,4.3205818893229075,4.237796624525191,4.145497098546789,4.0463930759081865,3.9436458660212845,3.840666601096081,3.7407788927879295,3.646868437851495,3.5611406094296543,3.4850045896021604,3.4190645799250117,3.363165863031277,3.316423654322972,3.2772116753915057,3.2431722072817477,3.211320383587341,3.1782914831477043,3.1407519903042207,3.0959642212677023,3.042334071598776,2.9794953397843256,2.9080729023778185,2.829400641500897,2.7452498357473916,2.6575926144014472,2.5684126235585367,2.4795657647388394,2.3926877801191893,2.3091420461160923,2.229999512393312,2.156042638090863,2.0877859000075074,2.025506582168317],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=7","opacity":1.0,"showlegend":true,"visible":false,"x":[2.4355048520970453],"y":[3.043692509144023],"type":"scatter"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=8","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-2.145826678237409,-2.1712756035251797,-2.2168613334834624,-2.273787189805308,-2.328093670420091,-2.363053123216667,-2.362268392836392,-2.3128848524913743,-2.208287798776912,-2.0497463620060974,-1.8466631944368224,-1.6153557591701402,-1.376575219678345,-1.1522050439270455,-0.9617250572035904,-0.8190487383157804,-0.7302603247210562,-0.6928681683491631,-0.6969046529991092,-0.7272796341503818,-0.766871958294535,-0.7998623409446359,-0.8143993881426501,-0.8037219324042154,-0.7661369530961435,-0.7040703795959227,-0.6224249315054571,-0.5266904527164715,-0.4212795468522057,-0.30851713288180393,-0.188558912337371,-0.06005040458084521,0.07897537311889646,0.22999184424914532,0.39305098242928693,0.5663862894177423,0.7456248780389347,0.9229532088892571,1.087524387226524,1.2269846486647742,1.3296310689047184,1.3866372015469612,1.3938153148813102,1.352519290201337,1.2695044988391242,1.1558061081364022,1.0249235915658192,0.8907460208288396,0.7653741639655672,0.6566549825058392],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=8","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.14575647028031466,-0.23759917923119428,-0.3339081008643409,-0.4204579294414509,-0.4869129553133137,-0.5293104594741934,-0.5504250867095511,-0.5579975600566792,-0.5615262384960078,-0.5687854078770093,-0.5832939670678263,-0.6035955724754397,-0.6245540162571649,-0.6401439024831782,-0.6466773217386089,-0.6452397651696998,-0.6423677266342879,-0.6485363117685714,-0.6747764671312821,-0.728531204646818,-0.8100828114137979,-0.9107176409491866,-1.013559999472955,-1.0973181818256001,-1.1415554110079509,-1.131376964856588,-1.0603523031804913,-0.931382003461872,-0.7553775863686281,-0.5483070160768203,-0.32770390357884915,-0.1094628596617957,0.09465455839421266,0.27773802142278936,0.43674973521405436,0.5706826253426875,0.6779967294180376,0.7545179469920182,0.7934252816822585,0.7872593991045718,0.7310648311096757,0.6254082992637202,0.47806505304258495,0.3036267416831149,0.12101024164980823,-0.050402262863672664,-0.194735295703524,-0.30234118148706557,-0.37076739071837883,-0.4045810161629615],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=8","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.7433530961269633,-0.7977442495441206,-0.8554487133881403,-0.8999578471146003,-0.9178487840158037,-0.9024876636150332,-0.8553689591402563,-0.7849052700227667,-0.7033569789420024,-0.623160783239526,-0.5539919024870905,-0.5014750164970153,-0.46772538668503694,-0.453149074154787,-0.4584567023317131,-0.4858365957954125,-0.538662129262023,-0.6193849904321934,-0.7260638548539259,-0.8495105945276642,-0.9725076932768442,-1.0717209496004283,-1.1220982029534643,-1.1026259163542838,-1.0016139044737393,-0.8202996925073682,-0.5734352998879226,-0.2860706214857709,0.01250132625989353,0.29566922870986345,0.5450204202697594,0.7532076007425139,0.9240890771899462,1.0694833746884858,1.203629577354328,1.3374351002505016,1.4736712213102967,1.6047046319700504,1.7141328715470179,1.781586197184994,1.7889345363166504,1.7258508460311277,1.5930436033000317,1.4023354661415541,1.1738296506687067,0.9313184686099325,0.6975692867801567,0.4910207699599814,0.3240133977608445,0.20114789030580427],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=8","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.09401237362835302,-0.2450261577716334,-0.50427927671561,-0.6250491210537042,-0.5977890735207273,-0.467250465988287,-0.31097175494715823,-0.2023617458885938,-0.17776729784840017,-0.22452966266910024,-0.2954599353985137,-0.3407708665443492,-0.3391840728202107,-0.3107701349332028,-0.3045546280169682,-0.3682111814244209,-0.5173211079448028,-0.7217437816746677,-0.91711951498645,-1.0360958500975384,-1.0423792286842204,-0.9487506209458256,-0.8092138137556648,-0.6898155507616863,-0.6335018925182405,-0.6374960532717511,-0.6555868425601152,-0.6243580052475524,-0.49880736349658594,-0.27784497825923515,-0.005982944051761363,0.24963439303621393,0.4344206454170927,0.5338597634748518,0.5783393793747329,0.6231483422016877,0.7146220621666303,0.86153421136032,1.0282742341192375,1.1537182667398789,1.1854284802545263,1.110281004539919,0.964445187215865,0.8168552527664641,0.7347970610675505,0.7500420126669645,0.8437197826196893,0.9579609168329634,1.027660480001373,1.0146693342699769],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=8","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.17531648525807053,0.01949505674206431,-0.11928153108593076,-0.23621399957027966,-0.32878410770448285,-0.39676911408830806,-0.44198183309693007,-0.46781635017377887,-0.4787021304980352,-0.47956765395024115,-0.47538998198618065,-0.47086458774275597,-0.4701794328869202,-0.47682900281533214,-0.4933671583959893,-0.5209784878246317,-0.5587960416926405,-0.6035557195940293,-0.6501583917982414,-0.6923094946751198,-0.7230645945519445,-0.7354470529691679,-0.7229841249320065,-0.6799090640752343,-0.6015163734862549,-0.4846917499135055,-0.3285216337154987,-0.13494258826692151,0.09073945089345459,0.3396290233154712,0.5994439415189845,0.8552824896826833,1.0910370319196652,1.2912002181118507,1.4428238561843936,1.5373531008775605,1.571730252371277,1.5486083580156813,1.4759221189003657,1.365840814929014,1.2331897016562618,1.0935324672984141,0.9611836992003213,0.847453929847438,0.7594150107988384,0.6994129026496018,0.6654591339603526,0.6524948300293961,0.6537896044310689,0.6611198064256332],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=8","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[7.25505299557851,7.137551682383661,7.003065586816941,6.850757197833216,6.680245524148064,6.491760340924872,6.286298647605154,6.065765746217712,5.83307632485728,5.592184327451336,5.348006307706342,5.1062047928424725,4.872810809967363,4.653694310285095,4.453943872902459,4.277295204125445,4.1258701171820285,4.000820440853111,3.9037247422528303,3.8373529148411953,3.8052959931501005,3.810574933351678,3.8541959386697613,3.934633645627583,4.047576706081331,4.186818119196312,4.346050724332167,4.52015823126421,4.705617420523785,4.899987273532085,5.100712445487534,5.304208480937758,5.506177597643976,5.702026516161326,5.886748949195261,6.054451127081881,6.198242314874243,6.310870077358628,6.385734591063956,6.417950774314065,6.405252403102528,6.348620080091091,6.252572137625074,6.125090798253921,5.977178017209505,5.822060393541326,5.67410182130605,5.547534687975744,5.455024839002489,5.405759381958511,4.277924512377107,4.46449311228622,4.616210139485795,4.730391078807945,4.808057387340481,4.854646701482398,4.878434025762911,4.888301415636657,4.89192192709032,4.894461110972527,4.897822756403338,4.900440855165586,4.89760854833719,4.882326422378506,4.846633699544046,4.783344583071062,4.688041952330253,4.560869038892449,4.406431335216315,4.232053896327058,4.0464433215186,3.8590580262815575,3.679628066453743,3.517018185795305,3.3779759398107783,3.266404455070053,3.1833198578428874,3.1274012229332975,3.095792161838991,3.0846330951460037,3.08922608254278,3.1039919010735906,3.1220928447985687,3.1352892952286235,3.1346636420994924,3.1124180221880318,3.0640731210223864,2.988994020737389,2.8895771002806843,2.7702234564161756,2.6363705655647864,2.4936984627503653,2.3475533670792377,2.202582875946002,2.0625503393526308,1.9302858513170351,1.8077314225398453,1.6960435364851105,1.5957241282641816,1.5067590267691822],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=8","opacity":1.0,"showlegend":true,"visible":false,"x":[4.5153938687938275],"y":[7.49092917848877],"type":"scatter"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=9","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[4.144492943879669,4.090193220872079,3.978057131299762,3.821084334869501,3.6257886338585377,3.3918107926034304,3.114199301704497,2.787500424963305,2.4102811207501915,1.9885911779624892,1.5371874779140122,1.0780088571701292,0.6362241230695951,0.23495058616091546,-0.10975759377903305,-0.3919640992124924,-0.6157103467076896,-0.7924157561022531,-0.9373649357622389,-1.064882703499369,-1.1835737704076295,-1.2936645921695078,-1.3863733990114835,-1.4440522022082223,-1.4433786108299893,-1.36181442153841,-1.1851528753457075,-0.9134274549247142,-0.5632805019508231,-0.16576269498039986,0.24034881938412678,0.6172202605509647,0.9349232149472566,1.1763846358642507,1.3390489943750645,1.4323583679533354,1.4743441027707702,1.4880182315723938,1.496051052888864,1.514592350925122,1.5472595230520874,1.5825398367505379,1.5957743977744046,1.5550769858421107,1.4296765819681339,1.1987524346740348,0.8589517334020798,0.4293214256910458,-0.04930350802318362,-0.5246443952557387],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=9","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-1.391044627799648,-1.2895448090628259,-1.1110252537650365,-0.8986067646637148,-0.6914480163814924,-0.514402121157961,-0.37409926294086904,-0.2624981725037464,-0.16577807851016446,-0.07427804641567426,0.011037438261401245,0.0783366339134568,0.11070902741782329,0.09501620384114506,0.02948362564720519,-0.07361859002459403,-0.19151123293581518,-0.29938453552244515,-0.3802727391734184,-0.43138702374767945,-0.4639134393558825,-0.49634939941915174,-0.5442087982127101,-0.6105122127332033,-0.6814911522494733,-0.7298149314032331,-0.7240790275001123,-0.6404562580462723,-0.47173597554301594,-0.23017251052145604,0.05672456475973575,0.35577954634790143,0.6395284337847528,0.8941392812789872,1.1209599555208773,1.331758925443731,1.5385108646132777,1.7422436542947493,1.926752050325279,2.0608499577409587,2.1097889997998673,2.048313979047582,1.869479699875925,1.5864036878509706,1.227084861515744,0.8254584064643102,0.41368862119627625,0.020351306673796365,-0.3279181058062819,-0.6111080552376973],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=9","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-1.3773695196993008,-1.2888290530801665,-1.141616268448984,-0.9534379283177139,-0.7447665121429752,-0.5365091701121691,-0.34766730777121446,-0.1933249014243823,-0.08325737657449328,-0.02135486314229329,-0.005918658708171209,-0.030739325946949725,-0.08672342789857201,-0.1637262097573312,-0.2521863327821766,-0.34415353450292774,-0.43341631626341653,-0.515453814501557,-0.5879979362031423,-0.6510763303329609,-0.706418458256893,-0.7566174725955047,-0.8038046665496413,-0.8473894374954348,-0.8821891523563017,-0.8985349835596861,-0.8841115394221859,-0.8263762621367179,-0.715317379875418,-0.5464582710109775,-0.32398009416194107,-0.061963092289310406,0.2187587154549111,0.495183353379574,0.746869636246636,0.9603308380040176,1.1298918016759085,1.2549746323538535,1.336492548165459,1.3742300635826794,1.3670596710654754,1.3134907972367555,1.2118073203772166,1.0602482562421698,0.8576852072719747,0.6051845272010521,0.3087588742174079,-0.01661160824005936,-0.34554148415887687,-0.6506391892466374],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=9","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-2.092858239797658,-2.029582609765767,-1.9323628775162818,-1.817892968358139,-1.70559344651911,-1.6128237938867578,-1.5502232687059343,-1.5185236470910444,-1.5078747290334884,-1.5001125797323254,-1.4736249606956267,-1.4097202209474542,-1.2988761204327695,-1.1450738497448185,-0.9666611765433126,-0.7927834283955533,-0.6554387872744796,-0.5809245574649178,-0.5848994245588179,-0.6692107511205987,-0.8213400269102475,-1.0180741137353304,-1.230253767674251,-1.423613835300035,-1.56059216965792,-1.6065629151517342,-1.5377676931741733,-1.3463716309638154,-1.0414552260823453,-0.6460232448963539,-0.19096532674147904,0.2917317873722318,0.7737721666279382,1.2329545173156742,1.6526354351858346,2.0185508877018483,2.3174745537624286,2.5386701824291964,2.675874661302832,2.7280992053551953,2.6972026810494762,2.586443955375371,2.4010912729263114,2.1500390629756083,1.8473388756533002,1.5128859806292168,1.1720984043908755,0.8550247415150558,0.5930086731791153,0.40885971414843336],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=9","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[1.5403688114477965,1.84927717264865,2.0943357428523806,2.256973817044829,2.3254251954578655,2.2964559471702555,2.1757304272067937,1.976730747677741,1.7184299369062972,1.4221660524101627,1.108323230351215,0.7934649200216678,0.4884791579801726,0.19810759332574035,-0.07801649543135714,-0.3429432277863873,-0.598984725513432,-0.8448380979652839,-1.074702261395493,-1.2783667798316989,-1.4420669801600297,-1.5502092213048666,-1.5880678547070817,-1.5454418230284124,-1.420199916219219,-1.2191714500436914,-0.9564476535431271,-0.650861787600725,-0.3229646676555089,0.008197962425850802,0.32802182338742547,0.6280847389585147,0.9057452578674015,1.1618866656637001,1.3969988065601628,1.6064966455085652,1.7798932334828446,1.9045142661788932,1.9709272525380288,1.9762648150861655,1.92126968040533,1.8075345665037246,1.6370814820649797,1.4134237451607077,1.1431670657172355,0.8373024559723362,0.5115404484503739,0.1853029549989536,-0.12067164392292196,-0.3878580251913796],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=9","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[11.883401498927746,11.573254841945296,11.20851176446407,10.786082907901966,10.304972518114692,9.767093835999152,9.178096534615705,8.54807856255988,7.891992991676303,7.229498674349257,6.583962127135022,5.980333264421032,5.441745503372584,4.985006319874859,4.61572525905208,4.324701015302489,4.0884432370181,3.879122547206477,3.680547872167468,3.49606637484856,3.3446804911744037,3.2489617573019305,3.2243070789518797,3.275593479788058,3.3948632582116396,3.5652703276372018,3.7718672056901674,4.009624090005277,4.283846949960568,4.602682166530621,4.965807435360566,5.3597719379391675,5.7663124255336555,6.1708233373105426,6.56318676148459,6.9333898929280195,7.271310212612353,7.568840826375143,7.815442774946349,7.9913773028052795,8.069261563787231,8.02238630996145,7.834103965504324,7.504568878390485,7.0530015235923145,6.515458291968082,5.939797516475663,5.38101865278652,4.898686620375203,4.549773016916674,3.768871669994112,4.394021480645078,4.954817275497169,5.439007237292357,5.838324794308706,6.159411659898872,6.42029596458346,6.639371407537654,6.825820541689008,6.974552452658624,7.067103955370733,7.078376513709,6.987396559801369,6.788468859066758,6.49519806109064,6.131165547477493,5.7210286055265716,5.289115433043287,4.857291799771986,4.439977187823161,4.044368927451473,3.6779138832482126,3.355840906463056,3.097496230288278,2.916404428502661,2.8134285552582767,2.7770841475352155,2.790642683167867,2.841283824678819,2.9231413925707246,3.035049989153327,3.1774241653316277,3.3437057301449995,3.5112149916757973,3.6399984447713774,3.683567199458939,3.6071713286401597,3.3969468560224847,3.0582657523272827,2.610462240490886,2.0807885771788714,1.4990946189779573,0.8938816623496026,0.28982384317906007,-0.2934469708099652,-0.841610836022979,-1.345306580370126,-1.7994290549558434,-2.202282143263047,-2.5547265688445977],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=9","opacity":1.0,"showlegend":true,"visible":false,"x":[5.035550221532578],"y":[8.35378853998293],"type":"scatter"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=10","showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.8033765164781723,-0.5120935476863505,-0.2854559108109338,-0.16235718473971628,-0.13229627248486242,-0.14373602840531285,-0.1324731856684288,-0.055927717703264546,0.08361387281615917,0.2383216171844319,0.3371427057084665,0.32026117145647337,0.16888529174284483,-0.08415215337523463,-0.36809097141795577,-0.6054660899102258,-0.7455892907720802,-0.7843672907411864,-0.7615478685350404,-0.7373582031274629,-0.7612206458506419,-0.8492853694449194,-0.9792881275976045,-1.0999421789271393,-1.152755155882749,-1.1012107159379987,-0.950445533903518,-0.7425976045078593,-0.5324664538866788,-0.3586034988209873,-0.2269252568314463,-0.11332915068888821,0.020878974810821155,0.20540236060911343,0.4432266124229125,0.711079118448677,0.970615154557075,1.182990073933039,1.3217852061148476,1.3797061730864872,1.3720740954598352,1.3251034184398933,1.2569186590203203,1.1645932670241508,1.0184904707367575,0.7743911921027169,0.40122157575285433,-0.09228232432984829,-0.6487307456386406,-1.1799209622457305],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=10","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-2.5200991720296027,-2.5297167642419947,-2.4389652802249158,-2.2606216846815963,-2.0143026453572586,-1.7242077973647736,-1.4165147024808749,-1.116760192541439,-0.847521931345118,-0.6266486788922329,-0.4661882189663913,-0.37204309229336907,-0.34426323284654237,-0.3777779423148303,-0.46329095671208936,-0.5880202051874475,-0.7360490965652706,-0.8892805454275996,-1.029956042992367,-1.1431315767715606,-1.2187603926792976,-1.2536332501765917,-1.2514341040105745,-1.2187641246501248,-1.1606906272166349,-1.079039325903162,-0.9730842563994652,-0.8408499548023977,-0.680733030976523,-0.4933277418972517,-0.28325930889468554,-0.05989048031535571,0.16449190596272345,0.3778135160647913,0.5722306628356122,0.7482228712137653,0.9132170468425373,1.0747040107485262,1.232540403671425,1.37492496096597,1.482448934084856,1.5334492634815386,1.508087607313807,1.3912814363774453,1.1743377464438929,0.8563165598700446,0.4461310831039387,-0.034194087560563524,-0.5493522469821017,-1.0618738183889351],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=10","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[0.5873177703197052,0.9018727023981237,1.1730549912528048,1.3639543593615469,1.449319835108538,1.4203317684961845,1.2853443976660954,1.0666914215879388,0.7947586138116782,0.5011782549962205,0.21301279869934042,-0.05077436099151411,-0.2801565293406041,-0.4726450407769951,-0.6306150694499002,-0.7583012254499453,-0.8590514190586056,-0.9345280911648057,-0.9867317592434528,-1.0192986338339023,-1.036507549416452,-1.0401437361214656,-1.0272098248085577,-0.9925936418812605,-0.9341818622045883,-0.8522422964925095,-0.7451824628340036,-0.6090472995475628,-0.44010179086260826,-0.2383013205272535,-0.00953463829141854,0.2339721396102119,0.4746544010862015,0.6936345827499764,0.878026577013591,1.0291245690347177,1.1608691305420362,1.2883692979848096,1.4151269962685986,1.526751618171866,1.5985209358865298,1.6052526345063693,1.5285304954382255,1.3608309507382508,1.1060877622756973,0.7780347741483509,0.39837101300764544,-0.003271798397475295,-0.3917691045253808,-0.7413106657100123],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=10","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-0.7549578663849379,-1.0551142496742842,-1.2976231163355982,-1.4666088547274923,-1.5529278779466065,-1.5552876691898558,-1.4804226942290937,-1.3423051070292096,-1.1605031674107114,-0.9579156454047442,-0.7581878078115629,-0.583141239192464,-0.45052080293458063,-0.3722809374826901,-0.3535113708784028,-0.3919562200920934,-0.47800689142498687,-0.5959458293383095,-0.7270427423404398,-0.8527665785381031,-0.9573953133045086,-1.0298975892450795,-1.0647259033856311,-1.0612094407140267,-1.0222278714603963,-0.9525511298009721,-0.8571666613917672,-0.7399993599550767,-0.6033212188008474,-0.4480689886372682,-0.27514333464181195,-0.08670259499930255,0.11392831420678506,0.32260659543256054,0.5347255000530295,0.7457362623912647,0.9506085207556034,1.142520644481996,1.31226958592306,1.4485104966812568,1.5386245326465593,1.5697444301713315,1.5295439209851498,1.4072146700549855,1.1961705272042549,0.8973652531409506,0.5224436224910471,0.09707733657534856,-0.3389461610116449,-0.7415911118905711],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"hoverinfo":"skip","marker":{"color":"red"},"mode":"lines","name":"a=10","showlegend":false,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0],"y":[-4.300522696085728,-4.262311470063608,-4.228491660652342,-4.166421704017141,-4.046212340247331,-3.8467209029527685,-3.5597294564401785,-3.1915928450719546,-2.7621609156545546,-2.3013153521223217,-1.8439044319553637,-1.424121577863076,-1.070411490233151,-0.8018074553713468,-0.6262590589156569,-0.5410881270907392,-0.5352728603751967,-0.5924522376625443,-0.6934568365606707,-0.8183762761855002,-0.9478639922759236,-1.0634483771612169,-1.1480011527315144,-1.1879618651439425,-1.1757571522840293,-1.110071606202006,-0.9945115265342328,-0.8364611963241999,-0.6459946137515269,-0.4343050857957613,-0.21119365598969253,0.016841746628329712,0.2455668087803473,0.4714883422376519,0.6903980844222386,0.8956700096741912,1.0792900489062727,1.2348037962556235,1.358461596080685,1.4482820880138845,1.5016743250844846,1.5132551268990935,1.474347600441747,1.3739119446758923,1.1982890240173507,0.9324031343851393,0.5642511861395826,0.09171861815654836,-0.4727204260404547,-1.1065122457357024],"type":"scatter","xaxis":"x2","yaxis":"y2"},{"fill":"toself","hoverinfo":"skip","name":"p=10","opacity":0.6,"showlegend":true,"visible":false,"x":[0.0,0.12244897959183673,0.24489795918367346,0.36734693877551017,0.4897959183673469,0.6122448979591837,0.7346938775510203,0.8571428571428571,0.9795918367346939,1.1020408163265305,1.2244897959183674,1.346938775510204,1.4693877551020407,1.5918367346938775,1.7142857142857142,1.836734693877551,1.9591836734693877,2.0816326530612246,2.204081632653061,2.326530612244898,2.4489795918367347,2.571428571428571,2.693877551020408,2.816326530612245,2.9387755102040813,3.061224489795918,3.183673469387755,3.306122448979592,3.4285714285714284,3.5510204081632653,3.673469387755102,3.7959183673469385,3.9183673469387754,4.040816326530612,4.163265306122449,4.285714285714286,4.408163265306122,4.530612244897959,4.653061224489796,4.775510204081632,4.8979591836734695,5.020408163265306,5.142857142857142,5.26530612244898,5.387755102040816,5.5102040816326525,5.63265306122449,5.755102040816326,5.877551020408163,6.0,6.0,5.877551020408163,5.755102040816326,5.63265306122449,5.5102040816326525,5.387755102040816,5.26530612244898,5.142857142857142,5.020408163265306,4.8979591836734695,4.775510204081632,4.653061224489796,4.530612244897959,4.408163265306122,4.285714285714286,4.163265306122449,4.040816326530612,3.9183673469387754,3.7959183673469385,3.673469387755102,3.5510204081632653,3.4285714285714284,3.306122448979592,3.183673469387755,3.061224489795918,2.9387755102040813,2.816326530612245,2.693877551020408,2.571428571428571,2.4489795918367347,2.326530612244898,2.204081632653061,2.0816326530612246,1.9591836734693877,1.836734693877551,1.7142857142857142,1.5918367346938775,1.4693877551020407,1.346938775510204,1.2244897959183674,1.1020408163265305,0.9795918367346939,0.8571428571428571,0.7346938775510203,0.6122448979591837,0.4897959183673469,0.36734693877551017,0.24489795918367346,0.12244897959183673,0.0],"y":[18.338065481744763,17.427615671766084,16.417416088487727,15.314346266669592,14.131239295945079,12.887686261920866,11.610414190349033,10.332968023486782,9.094391849782058,7.936615492508028,6.900349241934507,6.019524555368391,5.314751421396943,4.786940336256132,4.413144357826833,4.147638597404472,3.9320282966156146,3.718448064089602,3.4929510013171514,3.2760539397777237,3.101946373960347,2.9938601180304407,2.9593488796892826,3.001831963511977,3.11606858889696,3.282756326781617,3.4815572418422023,3.7070569264459867,3.9716959056203227,4.291780635037977,4.666937150514814,5.0761441392356526,5.498228455654141,5.926336384449383,6.360555550074437,6.792171443088367,7.2056939457578295,7.5890697725443514,7.927475476871721,8.195182254357563,8.366620651717986,8.425634938448406,8.355672721872622,8.125918341804226,7.70253432035509,7.07178966067408,6.25142602356157,5.291000342665118,4.274385793055321,3.3151812996124463,3.107659452193013,4.186558028721026,5.219185639111165,6.158336462449998,6.946806324758837,7.556395365172216,7.985256017671233,8.238498113646834,8.323438115594135,8.25969738170461,8.081521621768013,7.820586574725808,7.49481230500878,7.116557895132225,6.7031398412094,6.269521045280971,5.823543608736872,5.377473859490353,4.950743444044229,4.554252455990262,4.182838611307936,3.829427337239616,3.505278927036325,3.236444129571852,3.0437673064873065,2.9291425153662978,2.8783524198667463,2.87656332286926,2.920377208414964,3.0102576431327632,3.1423985267282357,3.317600976107925,3.5392277793343196,3.7935055210751942,4.03341221902894,4.183934530527716,4.169249629416374,3.935290302321866,3.4571568515507236,2.73804520897638,1.8035075280583799,0.6939290342417923,-0.5428279656997868,-1.8575464893554532,-3.2041362822354102,-4.542485271369539,-5.839915824302907,-7.071568354377181,-8.220038746668326,-9.274555360610885],"type":"scatter"},{"marker":{"size":10},"mode":"markers","name":"i=10","opacity":1.0,"showlegend":true,"visible":false,"x":[5.143971561084619],"y":[8.298211828582026],"type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,0.98],"matches":"x2","showticklabels":false,"range":[-1,7],"title":{"text":"X"}},"yaxis":{"anchor":"x","domain":[0.575,1.0]},"xaxis2":{"anchor":"y2","domain":[0.0,0.98],"range":[-1,7]},"yaxis2":{"anchor":"x2","domain":[0.0,0.425]},"annotations":[{"font":{"size":16},"showarrow":false,"text":"Surrogate","textangle":90,"x":0.98,"xanchor":"left","xref":"paper","y":0.7875,"yanchor":"middle","yref":"paper"},{"font":{"size":16},"showarrow":false,"text":"Acquisition","textangle":90,"x":0.98,"xanchor":"left","xref":"paper","y":0.2125,"yanchor":"middle","yref":"paper"}],"title":{"text":"Gold Search"},"sliders":[{"active":0,"steps":[{"args":[{"visible":[true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"0"}],"label":"i = 0","method":"update"},{"args":[{"visible":[true,true,true,true,true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"4"}],"label":"i = 0","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,false,false,false,false,true,true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"11"}],"label":"i = 1","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"18"}],"label":"i = 2","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"25"}],"label":"i = 3","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,true,true,true,true,true,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false],"title":"32"}],"label":"i = 4","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,true,true,true,true,true,true,true,false,false,false,false,false,false,false],"title":"39"}],"label":"i = 5","method":"update"},{"args":[{"visible":[true,true,true,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,true,true,true,true,true,true,true],"title":"46"}],"label":"i = 6","method":"update"}]}]},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Random
Choose the next point by uniformly sampling a point from the domain.

In [19]:
generation_strategy = create_generation_strategy(
    botorch_acqf_class=None,
    random_class=Models.UNIFORM,
    random_trials=10,
)
ax_client = create_client(generation_strategy=generation_strategy)
predictions, acquisitions = run_ax(n=10, ax_client=ax_client, acqf_n=0)
results["Random"] = ax_client.get_trials_data_frame()

[WARNING 08-10 23:19:27] ax.service.ax_client: Random seed set to 1. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 08-10 23:19:27] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x', parameter_type=FLOAT, range=[0.0, 6.0])], parameter_constraints=[]).
[WARNING 08-10 23:19:27] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


## Comparison
Compare the acquisition methods from above by showing the max gold achieved against how many trials in the experiment.

In [20]:
max_gold = {"Method": [], "Trial": [], "Max Gold": []}
for k, v in results.items():
    max_gold["Method"].extend([k] * (len(v) + 1))
    max_gold["Trial"].extend([0] + (v.trial_index + 1).to_list())
    max_gold["Max Gold"].extend([0] + v["gold"].cummax().tolist())
render_plotly_html(px.line(max_gold, x="Trial", y="Max Gold", color="Method"))

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.34.0.min.js"></script>                <div id="dff2f22b-bb9c-4571-a605-633fb9775456" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("dff2f22b-bb9c-4571-a605-633fb9775456")) {                    Plotly.newPlot(                        "dff2f22b-bb9c-4571-a605-633fb9775456",                        [{"hovertemplate":"Method=PI\u003cbr\u003eTrial=%{x}\u003cbr\u003eMax Gold=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"PI","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"PI","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10],"xaxis":"x","y":[0.0,4.153277647970732,6.4030358079544385,6.4030358079544385,7.78026814236398,7.78026814236398,7.78026814236398,7.97083381305886,8.353438586358578,8.353817966113354,8.353817966113354],"yaxis":"y","type":"scatter"},{"hovertemplate":"Method=EI\u003cbr\u003eTrial=%{x}\u003cbr\u003eMax Gold=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"EI","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"EI","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10],"xaxis":"x","y":[0.0,4.153277647970732,6.4030358079544385,6.4030358079544385,6.4030358079544385,6.4030358079544385,6.4030358079544385,6.909021517750929,7.691138525684522,8.331261799710944,8.348254087751911],"yaxis":"y","type":"scatter"},{"hovertemplate":"Method=UCB\u003cbr\u003eTrial=%{x}\u003cbr\u003eMax Gold=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"UCB","line":{"color":"#00cc96","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"UCB","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10],"xaxis":"x","y":[0.0,4.153277647970732,6.4030358079544385,6.4030358079544385,8.062511138469858,8.062511138469858,8.353042361423036,8.353042361423036,8.353042361423036,8.353604872575431,8.353772660864934],"yaxis":"y","type":"scatter"},{"hovertemplate":"Method=Thompson\u003cbr\u003eTrial=%{x}\u003cbr\u003eMax Gold=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Thompson","line":{"color":"#ab63fa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Thompson","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10],"xaxis":"x","y":[0.0,4.153277647970732,6.4030358079544385,6.4030358079544385,6.4030358079544385,6.4030358079544385,6.4030358079544385,6.4030358079544385,7.49092917848877,8.35378853998293,8.35378853998293],"yaxis":"y","type":"scatter"},{"hovertemplate":"Method=Random\u003cbr\u003eTrial=%{x}\u003cbr\u003eMax Gold=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Random","line":{"color":"#FFA15A","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Random","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10],"xaxis":"x","y":[0.0,5.057664805217066,5.091532753266253,5.703244208949001,5.776814007863983,5.776814007863983,5.776814007863983,5.776814007863983,5.776814007863983,6.01297425838335,6.01297425838335],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Trial"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"Max Gold"}},"legend":{"title":{"text":"Method"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>